# Calculate SMB, SEB values

Author: Ziqi Yin

Last modified: 2024 Dec 21

Description: Calculate SMB, SEB, and related variables averaged over years -19-0, 131-150, 231-250, 331-350.

In [28]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import xarray as xr
from scipy import stats

import cartopy
import cartopy.crs as ccrs
import numpy.ma as ma
import matplotlib.colors as colors
from statistics import mode
import pandas as pd
from scipy import signal
from cartopy.util import add_cyclic_point
from scipy import interpolate
import glob

home_dir = '/glade/work/ziqiyin/'
import sys  
sys.path.insert(0, home_dir)
sys.path.append(home_dir + 'function/')

from functions import *
#from Plot_2D import Plot_2D # To draw plots
#import matplotlib.cm as cm # To change colormap used in plots

crs_np = ccrs.NorthPolarStereo(central_longitude=0)
crs_ll = ccrs.PlateCarree()

# For unstructured grid
#import datashader as ds
#import datashader.utils as du
#import geoviews as gv
#import geoviews.feature as gf
#import holoviews as hv
#from datashader import reductions as rd, transfer_functions as tf
from holoviews.operation.datashader import datashade, rasterize, regrid
#from scipy.spatial import Delaunay
#hv.extension("bokeh")
import warnings
warnings.filterwarnings('ignore')

from matplotlib.collections import PolyCollection

# For regridding
import os
import ESMF
import xesmf as xe

# For creating ERA5 GrIS mask
import geopandas as gp
import matplotlib.patheffects as pe
import pooch
import regionmask

from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

import pyproj

In [3]:
## Parameter
g = 9.80616 #[m/s2]

## 1. SMB

### Read data

In [81]:
## BG iteration 7
BG7_RAIN_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/BG7/BG7_RAIN_yearly.nc')['RAIN'][:]
BG7_SNOW_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/BG7/BG7_SNOW_yearly.nc')['SNOW'][:]
BG7_QICE_MELT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/BG7/BG7_QICE_MELT_yearly.nc')['QICE_MELT'][:]
BG7_QSNOMELT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/BG7/BG7_QSNOMELT_yearly.nc')['QSNOMELT'][:]
#BG7_QFLX_EVAP_TOT_yearly = xr.open_dataset(home_dir+'dynamic_GrIS/BG7/BG7_QFLX_EVAP_TOT_yearly.nc')['QFLX_EVAP_TOT'][:] # Var not saved
BG7_QRUNOFF_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/BG7/BG7_QRUNOFF_yearly.nc')['QRUNOFF'][:]

In [82]:
## f09muntjewerf
# 1pctCO2
f09_1pctCO2_RAIN_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_RAIN_1pctCO2_yearly.nc')['RAIN'][:140]
f09_1pctCO2_SNOW_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_SNOW_1pctCO2_yearly.nc')['SNOW'][:140]
f09_1pctCO2_QICE_MELT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_QICE_MELT_1pctCO2_yearly.nc')['QICE_MELT'][:140]
f09_1pctCO2_QSNOMELT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_QSNOMELT_1pctCO2_yearly.nc')['QSNOMELT'][:140]
f09_1pctCO2_QFLX_EVAP_TOT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_QFLX_EVAP_TOT_1pctCO2_yearly.nc')['QFLX_EVAP_TOT'][:140]
f09_1pctCO2_QRUNOFF_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_QRUNOFF_1pctCO2_yearly.nc')['QRUNOFF'][:140]

# 4xCO2
f09_4xext_RAIN_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_RAIN_4xext_yearly.nc')['RAIN']
f09_4xext_SNOW_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_SNOW_4xext_yearly.nc')['SNOW']
f09_4xext_QICE_MELT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_QICE_MELT_4xext_yearly.nc')['QICE_MELT']
f09_4xext_QSNOMELT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_QSNOMELT_4xext_yearly.nc')['QSNOMELT']
f09_4xext_QFLX_EVAP_TOT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_QFLX_EVAP_TOT_4xext_yearly.nc')['QFLX_EVAP_TOT']
f09_4xext_QRUNOFF_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_QRUNOFF_4xext_yearly.nc')['QRUNOFF']

In [84]:
## f09nohacks
# piControl
f09_nohacks_piControl_RAIN_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_RAIN_piControl_yearly.nc')['RAIN'][:]
f09_nohacks_piControl_SNOW_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_SNOW_piControl_yearly.nc')['SNOW'][:]
f09_nohacks_piControl_QICE_MELT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_QICE_MELT_piControl_yearly.nc')['QICE_MELT'][:]
f09_nohacks_piControl_QSNOMELT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_QSNOMELT_piControl_yearly.nc')['QSNOMELT'][:]
f09_nohacks_piControl_QFLX_EVAP_TOT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_QFLX_EVAP_TOT_piControl_yearly.nc')['QFLX_EVAP_TOT'][:]
f09_nohacks_piControl_QRUNOFF_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_QRUNOFF_piControl_yearly.nc')['QRUNOFF'][:]

# 1pctCO2
f09_nohacks_1pctCO2_RAIN_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_RAIN_1pctCO2_yearly.nc')['RAIN'][:]
f09_nohacks_1pctCO2_SNOW_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_SNOW_1pctCO2_yearly.nc')['SNOW'][:]
f09_nohacks_1pctCO2_QICE_MELT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_QICE_MELT_1pctCO2_yearly.nc')['QICE_MELT'][:]
f09_nohacks_1pctCO2_QSNOMELT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_QSNOMELT_1pctCO2_yearly.nc')['QSNOMELT'][:]
f09_nohacks_1pctCO2_QFLX_EVAP_TOT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_QFLX_EVAP_TOT_1pctCO2_yearly.nc')['QFLX_EVAP_TOT'][:]
f09_nohacks_1pctCO2_QRUNOFF_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_QRUNOFF_1pctCO2_yearly.nc')['QRUNOFF'][:]

# 4xext
f09_nohacks_4xext_RAIN_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_RAIN_4xext_yearly.nc')['RAIN'][:]
f09_nohacks_4xext_SNOW_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_SNOW_4xext_yearly.nc')['SNOW'][:]
f09_nohacks_4xext_QICE_MELT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_QICE_MELT_4xext_yearly.nc')['QICE_MELT'][:]
f09_nohacks_4xext_QSNOMELT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_QSNOMELT_4xext_yearly.nc')['QSNOMELT'][:]
f09_nohacks_4xext_QFLX_EVAP_TOT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_QFLX_EVAP_TOT_4xext_yearly.nc')['QFLX_EVAP_TOT'][:]
f09_nohacks_4xext_QRUNOFF_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_QRUNOFF_4xext_yearly.nc')['QRUNOFF'][:]

In [85]:
## Arctic grid
# piControl
ARCTIC_piControl_p1_RAIN_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_RAIN_piControl_p1_yearly.nc')['RAIN']
ARCTIC_piControl_p2_RAIN_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_RAIN_piControl_p2_yearly.nc')['RAIN']
ARCTIC_piControl_p3_RAIN_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_RAIN_piControl_p3_yearly.nc')['RAIN']
ARCTIC_piControl_p4_RAIN_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_RAIN_piControl_p4_yearly.nc')['RAIN']
ARCTIC_piControl_p1_SNOW_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_SNOW_piControl_p1_yearly.nc')['SNOW']
ARCTIC_piControl_p2_SNOW_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_SNOW_piControl_p2_yearly.nc')['SNOW']
ARCTIC_piControl_p3_SNOW_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_SNOW_piControl_p3_yearly.nc')['SNOW']
ARCTIC_piControl_p4_SNOW_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_SNOW_piControl_p4_yearly.nc')['SNOW']
ARCTIC_piControl_p1_QICE_MELT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_QICE_MELT_piControl_p1_yearly.nc')['QICE_MELT']
ARCTIC_piControl_p2_QICE_MELT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_QICE_MELT_piControl_p2_yearly.nc')['QICE_MELT']
ARCTIC_piControl_p3_QICE_MELT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_QICE_MELT_piControl_p3_yearly.nc')['QICE_MELT']
ARCTIC_piControl_p4_QICE_MELT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_QICE_MELT_piControl_p4_yearly.nc')['QICE_MELT']
ARCTIC_piControl_p1_QSNOMELT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_QSNOMELT_piControl_p1_yearly.nc')['QSNOMELT']
ARCTIC_piControl_p2_QSNOMELT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_QSNOMELT_piControl_p2_yearly.nc')['QSNOMELT']
ARCTIC_piControl_p3_QSNOMELT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_QSNOMELT_piControl_p3_yearly.nc')['QSNOMELT']
ARCTIC_piControl_p4_QSNOMELT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_QSNOMELT_piControl_p4_yearly.nc')['QSNOMELT']
ARCTIC_piControl_p4_QFLX_EVAP_TOT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_QFLX_EVAP_TOT_piControl_p4_yearly.nc')['QFLX_EVAP_TOT']
ARCTIC_piControl_p4_QRUNOFF_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_QRUNOFF_piControl_p4_yearly.nc')['QRUNOFF']

# 1pctCO2
ARCTIC_1pctCO2_RAIN_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_RAIN_1pctCO2_yearly.nc')['RAIN'] 
ARCTIC_1pctCO2_SNOW_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_SNOW_1pctCO2_yearly.nc')['SNOW'] 
ARCTIC_1pctCO2_QICE_MELT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_QICE_MELT_1pctCO2_yearly.nc')['QICE_MELT'] 
ARCTIC_1pctCO2_QSNOMELT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_QSNOMELT_1pctCO2_yearly.nc')['QSNOMELT']
ARCTIC_1pctCO2_QFLX_EVAP_TOT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_QFLX_EVAP_TOT_1pctCO2_yearly.nc')['QFLX_EVAP_TOT'] 
ARCTIC_1pctCO2_QRUNOFF_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_QRUNOFF_1pctCO2_yearly.nc')['QRUNOFF']

# 4xext
ARCTIC_4xext_RAIN_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_RAIN_4xext_yearly.nc')['RAIN']
ARCTIC_4xext_SNOW_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_SNOW_4xext_yearly.nc')['SNOW']
ARCTIC_4xext_QICE_MELT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_QICE_MELT_4xext_yearly.nc')['QICE_MELT']
ARCTIC_4xext_QSNOMELT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_QSNOMELT_4xext_yearly.nc')['QSNOMELT']
ARCTIC_4xext_QFLX_EVAP_TOT_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_QFLX_EVAP_TOT_4xext_yearly.nc')['QFLX_EVAP_TOT']
ARCTIC_4xext_QRUNOFF_yearly = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_QRUNOFF_4xext_yearly.nc')['QRUNOFF']

In [86]:
## Area and land fraction
Re = 6371.22 * 1e3 #[m]

f09_mask_file1 = '/glade/p/cesmdata/cseg/inputdata/share/domains/domain.lnd.fv0.9x1.25_gx1v7.151020.nc'
f09_area = xr.open_dataset(f09_mask_file1)['area'] #[Radian2]
f09_area = f09_area.values * Re**2

f09_mask_file2 = '/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850G.f09_g17_gl4.CMIP6-1pctCO2to4x-withism.001/lnd/proc/tseries/month_1/b.e21.B1850G.f09_g17_gl4.CMIP6-1pctCO2to4x-withism.001.clm2.h0.RAIN.000101-005012.nc'
f09_land_frac = xr.open_dataset(f09_mask_file2)['landfrac'] #fraction of grid cell that is active

ARCTIC_mask_file1 = '/glade/p/cesmdata/cseg/inputdata/share/domains/domain.lnd.ne0np4.ARCTIC.ne30x4_tx0.1v2.200626.nc'
ARCTIC_area = xr.open_dataset(ARCTIC_mask_file1)['area'] #[Radian2]
ARCTIC_area = ARCTIC_area.values * Re**2

ARCTIC_mask_file2 = '/glade/campaign/cesm/development/amwg/aherring/cesm2_2_alpha06d.poparctic_B1850G1PCT_ne0ARCTICne30x4_g17_8192pes_220705_Nx2yrs.001/lnd/hist/cesm2_2_alpha06d.poparctic_B1850G1PCT_ne0ARCTICne30x4_g17_8192pes_220705_Nx2yrs.001.clm2.h0.0001-01.nc' #directly from clm2.h0 files
ARCTIC_land_frac = xr.open_dataset(ARCTIC_mask_file2)['landfrac']

In [7]:
## Mask
# f09
# Generate Greenland mask
#f09_mask_file1 = '/glade/work/aherring/grids/uniform-res/f09/clm_surfdata_5_0/surfdata_f09_hist_78pfts_CMIP6_simyr1979_c200430.nc'
#f09_mask_file1 = '/glade/work/aherring/grids/uniform-res/f09/clm_surfdata_5_0/surfdata_0.9x1.25_hist_78pfts_CMIP5_simyr1850_c200226.nc'
f09_mask_file1 = '/glade/work/aherring/grids/uniform-res/f09/clm_surfdata_5_0/surfdata_f09_hist_78pfts_CMIP6_simyr1850_c200429.nc'
f09_GrIS_mask0 = xr.open_dataset(f09_mask_file1)['GLACIER_REGION']
f09_GrIS_mask0 = xr.where(f09_GrIS_mask0==2, 1, np.nan)
# Glacier fraction of land area
f09_glacier_frac_piControl = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_PCT_LANDUNIT_piControl_yearly.nc')['PCT_LANDUNIT'][:,3,:,:]
f09_mask_file2 = '/glade/campaign/cgd/amp/aherring/archive/cam6_2_022.se_FHIST_f09_f09_mg17_1800pes_200507_mg3-Nx5yrs/lnd/hist/cam6_2_022.se_FHIST_f09_f09_mg17_1800pes_200507_mg3-Nx5yrs.clm2.h0.1979-01.nc'
f09_glacier_frac_const = xr.open_dataset(f09_mask_file2)['PCT_LANDUNIT'][0,3,:,:] #ltype_landice_multiple_elevation_classes
f09_glacier_frac_1pctCO2 = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_PCT_LANDUNIT_1pctCO2_yearly.nc')['PCT_LANDUNIT'][:140,3,:,:]
f09_glacier_frac_4xext = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_PCT_LANDUNIT_4xext_yearly.nc')['PCT_LANDUNIT'][:,3,:,:]
f09_glacier_frac = xr.concat([f09_glacier_frac_1pctCO2[:140], f09_glacier_frac_4xext], dim='time')
# Land fraction of grid cell
#f09_mask_file3 = '/glade/p/cesmdata/cseg/inputdata/share/domains/domain.lnd.fv0.9x1.25_gx1v7.151020.nc'
f09_mask_file3 = '/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850G.f09_g17_gl4.CMIP6-1pctCO2to4x-withism.001/lnd/proc/tseries/month_1/b.e21.B1850G.f09_g17_gl4.CMIP6-1pctCO2to4x-withism.001.clm2.h0.RAIN.000101-005012.nc'
f09_land_frac = xr.open_dataset(f09_mask_file3)['landfrac'] #fraction of grid cell that is active
f09_GrIS_mask_piControl = f09_GrIS_mask0.values * f09_glacier_frac_piControl.values * f09_land_frac.values / 100
f09_GrIS_mask_const = f09_GrIS_mask0.values * f09_glacier_frac_const.values * f09_land_frac.values / 100
f09_GrIS_mask = f09_GrIS_mask0.values * f09_glacier_frac.values * f09_land_frac.values / 100

# f09nohacks
f09_nohacks_glacier_frac_piControl = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_PCT_LANDUNIT_piControl_yearly.nc')['PCT_LANDUNIT'][:,3,:,:]
f09_nohacks_GrIS_mask_piControl = f09_GrIS_mask0.values * f09_nohacks_glacier_frac_piControl.values * f09_land_frac.values / 100
f09_nohacks_glacier_frac_1pctCO2 = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_PCT_LANDUNIT_1pctCO2_yearly.nc')['PCT_LANDUNIT'][:,3,:,:]
f09_nohacks_glacier_frac_4xext = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_PCT_LANDUNIT_4xext_yearly.nc')['PCT_LANDUNIT'][:,3,:,:]
f09_nohacks_glacier_frac = xr.concat([f09_nohacks_glacier_frac_1pctCO2, f09_nohacks_glacier_frac_4xext], dim='time')
f09_nohacks_GrIS_mask = f09_GrIS_mask0.values * f09_nohacks_glacier_frac.values * f09_land_frac.values / 100

# BG7
BG7_glacier_frac = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/BG7/BG7_PCT_LANDUNIT_yearly.nc')['PCT_LANDUNIT'][:,3,:,:]
BG7_GrIS_mask = f09_GrIS_mask0.values * BG7_glacier_frac.values * f09_land_frac.values / 100

# ARCTIC
# Generate Greenland mask
#ARCTIC_mask_file1 = '/glade/work/aherring/grids/var-res/ne0np4.ARCTIC.ne30x4/clm_surfdata_5_0/surfdata_ne0np4.ARCTIC.ne30x4_hist_78pfts_CMIP6_simyr2000_c200426.nc' #get same result
ARCTIC_mask_file1 = '/glade/work/aherring/grids/var-res/ne0np4.ARCTIC.ne30x4/clm_surfdata_5_0/surfdata_ne0np4.ARCTIC.ne30x4_hist_78pfts_CMIP6_simyr1850_c200426.nc'
ARCTIC_GrIS_mask0 = xr.open_dataset(ARCTIC_mask_file1)['GLACIER_REGION']
ARCTIC_GrIS_mask0 = xr.where(ARCTIC_GrIS_mask0==2, 1, np.nan)
# Glacier fraction of land area
ARCTIC_glacier_frac_piControl_p1 = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_PCT_LANDUNIT_piControl_p1_yearly.nc')['PCT_LANDUNIT'][:,3,:]
ARCTIC_glacier_frac_piControl_p2 = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_PCT_LANDUNIT_piControl_p2_yearly.nc')['PCT_LANDUNIT'][:,3,:]
ARCTIC_glacier_frac_piControl_p3 = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_PCT_LANDUNIT_piControl_p3_yearly.nc')['PCT_LANDUNIT'][:,3,:]
ARCTIC_glacier_frac_piControl_p4 = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_PCT_LANDUNIT_piControl_p4_yearly.nc')['PCT_LANDUNIT'][:,3,:]
ARCTIC_glacier_frac_piControl = xr.concat([ARCTIC_glacier_frac_piControl_p1, ARCTIC_glacier_frac_piControl_p2, ARCTIC_glacier_frac_piControl_p3, ARCTIC_glacier_frac_piControl_p4], dim='time')
ARCTIC_mask_file2 = '/glade/campaign/cgd/amp/aherring/archive/cam6_2_022.se_FHIST_ne0np4.ARCTIC.ne30x4_mt12_7680pes_200507_mg3-Nx2yrs/lnd/hist/cam6_2_022.se_FHIST_ne0np4.ARCTIC.ne30x4_mt12_7680pes_200507_mg3-Nx2yrs.clm2.h0.1979-01.nc'
ARCTIC_glacier_frac_const = xr.open_dataset(ARCTIC_mask_file2)['PCT_LANDUNIT'][0,3,:]
ARCTIC_glacier_frac_1pctCO2 = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_PCT_LANDUNIT_1pctCO2_yearly.nc')['PCT_LANDUNIT'][:,3,:]
ARCTIC_glacier_frac_4xext = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_PCT_LANDUNIT_4xext_yearly.nc')['PCT_LANDUNIT'][:,3,:]
ARCTIC_glacier_frac = xr.concat([ARCTIC_glacier_frac_1pctCO2, ARCTIC_glacier_frac_4xext], dim='time')
# Land fraction of grid cell
#ARCTIC_mask_file3 = '/glade/p/cesmdata/cseg/inputdata/share/domains/domain.lnd.ne0np4.ARCTIC.ne30x4_tx0.1v2.200626.nc' #Wrong domain, it is for B case
#ARCTIC_mask_file3 = '/glade/work/aherring/grids/var-res/ne0np4.ARCTIC.ne30x4/domains/domain.lnd.ne0np4.ARCTIC.ne30x4_gx1v7.200817.nc' #domain file with ['frac']
ARCTIC_mask_file3 = '/glade/campaign/cesm/development/amwg/aherring/cesm2_2_alpha06d.poparctic_B1850G1PCT_ne0ARCTICne30x4_g17_8192pes_220705_Nx2yrs.001/lnd/hist/cesm2_2_alpha06d.poparctic_B1850G1PCT_ne0ARCTICne30x4_g17_8192pes_220705_Nx2yrs.001.clm2.h0.0001-01.nc' #directly from clm2.h0 files
ARCTIC_land_frac = xr.open_dataset(ARCTIC_mask_file3)['landfrac']
ARCTIC_GrIS_mask_piControl = ARCTIC_GrIS_mask0.values * ARCTIC_glacier_frac_piControl.values * ARCTIC_land_frac.values / 100
ARCTIC_GrIS_mask_const = ARCTIC_GrIS_mask0.values * ARCTIC_glacier_frac_const.values * ARCTIC_land_frac.values / 100
ARCTIC_GrIS_mask = ARCTIC_GrIS_mask0.values * ARCTIC_glacier_frac.values * ARCTIC_land_frac.values / 100
#ARCTIC_GrIS_mask_v2 = ARCTIC_GrIS_mask0.values * ARCTIC_glacier_frac.values / 100

In [87]:
## Concat
f09_1pctCO2_4xext_RAIN_yearly = xr.concat([f09_1pctCO2_RAIN_yearly, f09_4xext_RAIN_yearly], dim='time')
f09_1pctCO2_4xext_SNOW_yearly = xr.concat([f09_1pctCO2_SNOW_yearly, f09_4xext_SNOW_yearly], dim='time')
f09_1pctCO2_4xext_QICE_MELT_yearly = xr.concat([f09_1pctCO2_QICE_MELT_yearly, f09_4xext_QICE_MELT_yearly], dim='time')
f09_1pctCO2_4xext_QSNOMELT_yearly = xr.concat([f09_1pctCO2_QSNOMELT_yearly, f09_4xext_QSNOMELT_yearly], dim='time')
f09_1pctCO2_4xext_QFLX_EVAP_TOT_yearly = xr.concat([f09_1pctCO2_QFLX_EVAP_TOT_yearly, f09_4xext_QFLX_EVAP_TOT_yearly], dim='time')
f09_1pctCO2_4xext_QRUNOFF_yearly = xr.concat([f09_1pctCO2_QRUNOFF_yearly, f09_4xext_QRUNOFF_yearly], dim='time')

f09_nohacks_1pctCO2_4xext_RAIN_yearly = xr.concat([f09_nohacks_1pctCO2_RAIN_yearly, f09_nohacks_4xext_RAIN_yearly], dim='time')
f09_nohacks_1pctCO2_4xext_SNOW_yearly = xr.concat([f09_nohacks_1pctCO2_SNOW_yearly, f09_nohacks_4xext_SNOW_yearly], dim='time')
f09_nohacks_1pctCO2_4xext_QICE_MELT_yearly = xr.concat([f09_nohacks_1pctCO2_QICE_MELT_yearly, f09_nohacks_4xext_QICE_MELT_yearly], dim='time')
f09_nohacks_1pctCO2_4xext_QSNOMELT_yearly = xr.concat([f09_nohacks_1pctCO2_QSNOMELT_yearly, f09_nohacks_4xext_QSNOMELT_yearly], dim='time')
f09_nohacks_1pctCO2_4xext_QFLX_EVAP_TOT_yearly = xr.concat([f09_nohacks_1pctCO2_QFLX_EVAP_TOT_yearly, f09_nohacks_4xext_QFLX_EVAP_TOT_yearly], dim='time')
f09_nohacks_1pctCO2_4xext_QRUNOFF_yearly = xr.concat([f09_nohacks_1pctCO2_QRUNOFF_yearly, f09_nohacks_4xext_QRUNOFF_yearly], dim='time')

ARCTIC_piControl_RAIN_yearly = xr.concat([ARCTIC_piControl_p1_RAIN_yearly, ARCTIC_piControl_p2_RAIN_yearly, ARCTIC_piControl_p3_RAIN_yearly, ARCTIC_piControl_p4_RAIN_yearly], dim='time')
ARCTIC_piControl_SNOW_yearly = xr.concat([ARCTIC_piControl_p1_SNOW_yearly, ARCTIC_piControl_p2_SNOW_yearly, ARCTIC_piControl_p3_SNOW_yearly, ARCTIC_piControl_p4_SNOW_yearly], dim='time')
ARCTIC_piControl_QICE_MELT_yearly = xr.concat([ARCTIC_piControl_p1_QICE_MELT_yearly, ARCTIC_piControl_p2_QICE_MELT_yearly, ARCTIC_piControl_p3_QICE_MELT_yearly, ARCTIC_piControl_p4_QICE_MELT_yearly], dim='time')
ARCTIC_piControl_QSNOMELT_yearly = xr.concat([ARCTIC_piControl_p1_QSNOMELT_yearly, ARCTIC_piControl_p2_QSNOMELT_yearly, ARCTIC_piControl_p3_QSNOMELT_yearly, ARCTIC_piControl_p4_QSNOMELT_yearly], dim='time')
ARCTIC_1pctCO2_4xext_RAIN_yearly = xr.concat([ARCTIC_1pctCO2_RAIN_yearly, ARCTIC_4xext_RAIN_yearly], dim='time')
ARCTIC_1pctCO2_4xext_SNOW_yearly = xr.concat([ARCTIC_1pctCO2_SNOW_yearly, ARCTIC_4xext_SNOW_yearly], dim='time')
ARCTIC_1pctCO2_4xext_QICE_MELT_yearly = xr.concat([ARCTIC_1pctCO2_QICE_MELT_yearly, ARCTIC_4xext_QICE_MELT_yearly], dim='time')
ARCTIC_1pctCO2_4xext_QSNOMELT_yearly = xr.concat([ARCTIC_1pctCO2_QSNOMELT_yearly, ARCTIC_4xext_QSNOMELT_yearly], dim='time')
ARCTIC_1pctCO2_4xext_QFLX_EVAP_TOT_yearly = xr.concat([ARCTIC_1pctCO2_QFLX_EVAP_TOT_yearly, ARCTIC_4xext_QFLX_EVAP_TOT_yearly], dim='time')
ARCTIC_1pctCO2_4xext_QRUNOFF_yearly = xr.concat([ARCTIC_1pctCO2_QRUNOFF_yearly, ARCTIC_4xext_QRUNOFF_yearly], dim='time')

### Calculate

In [88]:
## Calculate total precip, melt
BG7_prec_yearly = BG7_RAIN_yearly + BG7_SNOW_yearly
BG7_melt_yearly = BG7_QICE_MELT_yearly + BG7_QSNOMELT_yearly

f09_1pctCO2_4xext_prec_yearly = f09_1pctCO2_4xext_RAIN_yearly + f09_1pctCO2_4xext_SNOW_yearly
f09_1pctCO2_4xext_melt_yearly = f09_1pctCO2_4xext_QICE_MELT_yearly + f09_1pctCO2_4xext_QSNOMELT_yearly

f09_nohacks_piControl_prec_yearly = f09_nohacks_piControl_RAIN_yearly + f09_nohacks_piControl_SNOW_yearly
f09_nohacks_piControl_melt_yearly = f09_nohacks_piControl_QICE_MELT_yearly + f09_nohacks_piControl_QSNOMELT_yearly
f09_nohacks_1pctCO2_4xext_prec_yearly = f09_nohacks_1pctCO2_4xext_RAIN_yearly + f09_nohacks_1pctCO2_4xext_SNOW_yearly
f09_nohacks_1pctCO2_4xext_melt_yearly = f09_nohacks_1pctCO2_4xext_QICE_MELT_yearly + f09_nohacks_1pctCO2_4xext_QSNOMELT_yearly

ARCTIC_piControl_prec_yearly = ARCTIC_piControl_RAIN_yearly + ARCTIC_piControl_SNOW_yearly
ARCTIC_piControl_melt_yearly = ARCTIC_piControl_QICE_MELT_yearly + ARCTIC_piControl_QSNOMELT_yearly
ARCTIC_1pctCO2_4xext_prec_yearly = ARCTIC_1pctCO2_4xext_RAIN_yearly + ARCTIC_1pctCO2_4xext_SNOW_yearly
ARCTIC_1pctCO2_4xext_melt_yearly = ARCTIC_1pctCO2_4xext_QICE_MELT_yearly + ARCTIC_1pctCO2_4xext_QSNOMELT_yearly

In [10]:
## Integrate over the GrIS
# F09M
BG7_int_RAIN = (BG7_RAIN_yearly * BG7_GrIS_mask * f09_area).sum(dim=['lat','lon'])*(3600*24*365)/1e12
BG7_int_SNOW = (BG7_SNOW_yearly * BG7_GrIS_mask * f09_area).sum(dim=['lat','lon'])*(3600*24*365)/1e12
BG7_int_prec = (BG7_prec_yearly * BG7_GrIS_mask * f09_area).sum(dim=['lat','lon'])*(3600*24*365)/1e12
BG7_int_melt = (BG7_melt_yearly * BG7_GrIS_mask * f09_area).sum(dim=['lat','lon'])*(3600*24*365)/1e12
#BG7_int_QFLX_EVAP_TOT = (BG7_QFLX_EVAP_TOT_yearly * BG7_GrIS_mask * f09_area).sum(dim=['lat','lon'])*(3600*24*365)/1e12
BG7_int_QRUNOFF = (BG7_QRUNOFF_yearly * BG7_GrIS_mask * f09_area).sum(dim=['lat','lon'])*(3600*24*365)/1e12
f09_int_1pctCO2_4xext_RAIN = (f09_1pctCO2_4xext_RAIN_yearly * f09_GrIS_mask * f09_area).sum(dim=['lat','lon'])*(3600*24*365)/1e12
f09_int_1pctCO2_4xext_SNOW = (f09_1pctCO2_4xext_SNOW_yearly * f09_GrIS_mask * f09_area).sum(dim=['lat','lon'])*(3600*24*365)/1e12
f09_int_1pctCO2_4xext_prec = (f09_1pctCO2_4xext_prec_yearly * f09_GrIS_mask * f09_area).sum(dim=['lat','lon'])*(3600*24*365)/1e12
f09_int_1pctCO2_4xext_melt = (f09_1pctCO2_4xext_melt_yearly * f09_GrIS_mask * f09_area).sum(dim=['lat','lon'])*(3600*24*365)/1e12
f09_int_1pctCO2_4xext_QFLX_EVAP_TOT = (f09_1pctCO2_4xext_QFLX_EVAP_TOT_yearly * f09_GrIS_mask * f09_area).sum(dim=['lat','lon'])*(3600*24*365)/1e12
f09_int_1pctCO2_4xext_QRUNOFF = (f09_1pctCO2_4xext_QRUNOFF_yearly * f09_GrIS_mask * f09_area).sum(dim=['lat','lon'])*(3600*24*365)/1e12

#F09
f09_nohacks_int_piControl_RAIN = (f09_nohacks_piControl_RAIN_yearly * f09_nohacks_GrIS_mask_piControl * f09_area).sum(dim=['lat','lon'])*(3600*24*365)/1e12
f09_nohacks_int_piControl_SNOW = (f09_nohacks_piControl_SNOW_yearly * f09_nohacks_GrIS_mask_piControl * f09_area).sum(dim=['lat','lon'])*(3600*24*365)/1e12
f09_nohacks_int_piControl_prec = (f09_nohacks_piControl_prec_yearly * f09_nohacks_GrIS_mask_piControl * f09_area).sum(dim=['lat','lon'])*(3600*24*365)/1e12
f09_nohacks_int_piControl_melt = (f09_nohacks_piControl_melt_yearly * f09_nohacks_GrIS_mask_piControl * f09_area).sum(dim=['lat','lon'])*(3600*24*365)/1e12
f09_nohacks_int_piControl_QFLX_EVAP_TOT = (f09_nohacks_piControl_QFLX_EVAP_TOT_yearly * f09_nohacks_GrIS_mask_piControl * f09_area).sum(dim=['lat','lon'])*(3600*24*365)/1e12
f09_nohacks_int_piControl_QRUNOFF = (f09_nohacks_piControl_QRUNOFF_yearly * f09_nohacks_GrIS_mask_piControl * f09_area).sum(dim=['lat','lon'])*(3600*24*365)/1e12

f09_nohacks_int_1pctCO2_4xext_RAIN = (f09_nohacks_1pctCO2_4xext_RAIN_yearly * f09_nohacks_GrIS_mask * f09_area).sum(dim=['lat','lon'])*(3600*24*365)/1e12
f09_nohacks_int_1pctCO2_4xext_SNOW = (f09_nohacks_1pctCO2_4xext_SNOW_yearly * f09_nohacks_GrIS_mask * f09_area).sum(dim=['lat','lon'])*(3600*24*365)/1e12
f09_nohacks_int_1pctCO2_4xext_prec = (f09_nohacks_1pctCO2_4xext_prec_yearly * f09_nohacks_GrIS_mask * f09_area).sum(dim=['lat','lon'])*(3600*24*365)/1e12
f09_nohacks_int_1pctCO2_4xext_melt = (f09_nohacks_1pctCO2_4xext_melt_yearly * f09_nohacks_GrIS_mask * f09_area).sum(dim=['lat','lon'])*(3600*24*365)/1e12
f09_nohacks_int_1pctCO2_4xext_QFLX_EVAP_TOT = (f09_nohacks_1pctCO2_4xext_QFLX_EVAP_TOT_yearly * f09_nohacks_GrIS_mask * f09_area).sum(dim=['lat','lon'])*(3600*24*365)/1e12
f09_nohacks_int_1pctCO2_4xext_QRUNOFF = (f09_nohacks_1pctCO2_4xext_QRUNOFF_yearly * f09_nohacks_GrIS_mask * f09_area).sum(dim=['lat','lon'])*(3600*24*365)/1e12

#ARCTIC
ARCTIC_int_piControl_RAIN = (ARCTIC_piControl_RAIN_yearly * ARCTIC_GrIS_mask_piControl * ARCTIC_area).sum(dim=['lndgrid'])*(3600*24*365)/1e12
ARCTIC_int_piControl_SNOW = (ARCTIC_piControl_SNOW_yearly * ARCTIC_GrIS_mask_piControl * ARCTIC_area).sum(dim=['lndgrid'])*(3600*24*365)/1e12
ARCTIC_int_piControl_prec = (ARCTIC_piControl_prec_yearly * ARCTIC_GrIS_mask_piControl * ARCTIC_area).sum(dim=['lndgrid'])*(3600*24*365)/1e12
ARCTIC_int_piControl_melt = (ARCTIC_piControl_melt_yearly * ARCTIC_GrIS_mask_piControl * ARCTIC_area).sum(dim=['lndgrid'])*(3600*24*365)/1e12
ARCTIC_int_piControl_p4_QFLX_EVAP_TOT = (ARCTIC_piControl_p4_QFLX_EVAP_TOT_yearly * ARCTIC_GrIS_mask_piControl[-80:,:] * ARCTIC_area).sum(dim=['lndgrid'])*(3600*24*365)/1e12
ARCTIC_int_piControl_p4_QRUNOFF = (ARCTIC_piControl_p4_QRUNOFF_yearly * ARCTIC_GrIS_mask_piControl[-80:,:] * ARCTIC_area).sum(dim=['lndgrid'])*(3600*24*365)/1e12

ARCTIC_int_1pctCO2_4xext_RAIN = (ARCTIC_1pctCO2_4xext_RAIN_yearly * ARCTIC_GrIS_mask * ARCTIC_area).sum(dim=['lndgrid'])*(3600*24*365)/1e12
ARCTIC_int_1pctCO2_4xext_SNOW = (ARCTIC_1pctCO2_4xext_SNOW_yearly * ARCTIC_GrIS_mask * ARCTIC_area).sum(dim=['lndgrid'])*(3600*24*365)/1e12
ARCTIC_int_1pctCO2_4xext_prec = (ARCTIC_1pctCO2_4xext_prec_yearly * ARCTIC_GrIS_mask * ARCTIC_area).sum(dim=['lndgrid'])*(3600*24*365)/1e12
ARCTIC_int_1pctCO2_4xext_melt = (ARCTIC_1pctCO2_4xext_melt_yearly * ARCTIC_GrIS_mask * ARCTIC_area).sum(dim=['lndgrid'])*(3600*24*365)/1e12
ARCTIC_int_1pctCO2_4xext_QFLX_EVAP_TOT = (ARCTIC_1pctCO2_4xext_QFLX_EVAP_TOT_yearly * ARCTIC_GrIS_mask * ARCTIC_area).sum(dim=['lndgrid'])*(3600*24*365)/1e12
ARCTIC_int_1pctCO2_4xext_QRUNOFF = (ARCTIC_1pctCO2_4xext_QRUNOFF_yearly * ARCTIC_GrIS_mask * ARCTIC_area).sum(dim=['lndgrid'])*(3600*24*365)/1e12

### Result

In [16]:
# Snow
print('Snow')
print('')

print('F09M(P0): ', BG7_int_SNOW[129:149].mean().round(2).values)
print('F09(P0): ', f09_nohacks_int_piControl_SNOW[-20:].mean().round(2).values)
print('ARCTIC(P0): ', ARCTIC_int_piControl_SNOW[-20:].mean().round(2).values)
print('')

print('F09M(P1): ', f09_int_1pctCO2_4xext_SNOW[130:150].mean().round(2).values)
print('F09(P1): ', f09_nohacks_int_1pctCO2_4xext_SNOW[130:150].mean().round(2).values)
print('ARCTIC(P1): ', ARCTIC_int_1pctCO2_4xext_SNOW[130:150].mean().round(2).values)
print('')

print('F09M(P2): ', f09_int_1pctCO2_4xext_SNOW[230:250].mean().round(2).values)
print('F09(P2): ', f09_nohacks_int_1pctCO2_4xext_SNOW[230:250].mean().round(2).values)
print('ARCTIC(P2): ', ARCTIC_int_1pctCO2_4xext_SNOW[230:250].mean().round(2).values)
print('')

print('F09M(P3): ', f09_int_1pctCO2_4xext_SNOW[330:350].mean().round(2).values)
print('F09(P3): ', f09_nohacks_int_1pctCO2_4xext_SNOW[330:350].mean().round(2).values)
print('ARCTIC(P3): ', ARCTIC_int_1pctCO2_4xext_SNOW[330:350].mean().round(2).values)

Snow

F09M(P0):  849.99
F09(P0):  934.21
ARCTIC(P0):  869.43

F09M(P1):  781.77
F09(P1):  961.29
ARCTIC(P1):  929.63

F09M(P2):  699.17
F09(P2):  931.72
ARCTIC(P2):  912.93

F09M(P3):  695.26
F09(P3):  819.1
ARCTIC(P3):  849.41


In [15]:
# Rain
print('Rain')
print('')

print('F09M(P0): ', BG7_int_RAIN[129:149].mean().round(2).values)
print('F09(P0): ', f09_nohacks_int_piControl_RAIN[-20:].mean().round(2).values)
print('ARCTIC(P0): ', ARCTIC_int_piControl_RAIN[-20:].mean().round(2).values)
print('')

print('F09M(P1): ', f09_int_1pctCO2_4xext_RAIN[130:150].mean().round(2).values)
print('F09(P1): ', f09_nohacks_int_1pctCO2_4xext_RAIN[130:150].mean().round(2).values)
print('ARCTIC(P1): ', ARCTIC_int_1pctCO2_4xext_RAIN[130:150].mean().round(2).values)
print('')

print('F09M(P2): ', f09_int_1pctCO2_4xext_RAIN[230:250].mean().round(2).values)
print('F09(P2): ', f09_nohacks_int_1pctCO2_4xext_RAIN[230:250].mean().round(2).values)
print('ARCTIC(P2): ', ARCTIC_int_1pctCO2_4xext_RAIN[230:250].mean().round(2).values)
print('')

print('F09M(P3): ', f09_int_1pctCO2_4xext_RAIN[330:350].mean().round(2).values)
print('F09(P3): ', f09_nohacks_int_1pctCO2_4xext_RAIN[330:350].mean().round(2).values)
print('ARCTIC(P3): ', ARCTIC_int_1pctCO2_4xext_RAIN[330:350].mean().round(2).values)

Rain

F09M(P0):  91.91
F09(P0):  92.13
ARCTIC(P0):  85.49

F09M(P1):  265.49
F09(P1):  311.65
ARCTIC(P1):  270.04

F09M(P2):  406.34
F09(P2):  442.71
ARCTIC(P2):  363.69

F09M(P3):  460.98
F09(P3):  488.46
ARCTIC(P3):  415.56


In [11]:
# Precipitation
print('Precipitation')
print('')

print('F09M(P0): ', BG7_int_prec[129:149].mean().round(2).values)
print('F09(P0): ', f09_nohacks_int_piControl_prec[-20:].mean().round(2).values)
print('ARCTIC(P0): ', ARCTIC_int_piControl_prec[-20:].mean().round(2).values)
print('')

print('F09M(P1): ', f09_int_1pctCO2_4xext_prec[130:150].mean().round(2).values)
print('F09(P1): ', f09_nohacks_int_1pctCO2_4xext_prec[130:150].mean().round(2).values)
print('ARCTIC(P1): ', ARCTIC_int_1pctCO2_4xext_prec[130:150].mean().round(2).values)
print('')

print('F09M(P2): ', f09_int_1pctCO2_4xext_prec[230:250].mean().round(2).values)
print('F09(P2): ', f09_nohacks_int_1pctCO2_4xext_prec[230:250].mean().round(2).values)
print('ARCTIC(P2): ', ARCTIC_int_1pctCO2_4xext_prec[230:250].mean().round(2).values)
print('')

print('F09M(P3): ', f09_int_1pctCO2_4xext_prec[330:350].mean().round(2).values)
print('F09(P3): ', f09_nohacks_int_1pctCO2_4xext_prec[330:350].mean().round(2).values)
print('ARCTIC(P3): ', ARCTIC_int_1pctCO2_4xext_prec[330:350].mean().round(2).values)

Precipitation

F09M(P0):  941.9
F09(P0):  1026.33
ARCTIC(P0):  954.92

F09M(P1):  1047.25
F09(P1):  1272.94
ARCTIC(P1):  1199.67

F09M(P2):  1105.51
F09(P2):  1374.43
ARCTIC(P2):  1276.62

F09M(P3):  1156.24
F09(P3):  1307.56
ARCTIC(P3):  1264.97


In [18]:
# Rain(%)
print('Rain(%)')
print('')

print('F09M(P0): ', (100 * BG7_int_RAIN[129:149].mean() / BG7_int_prec[129:149].mean()).round(2).values)
print('F09(P0): ', (100 * f09_nohacks_int_piControl_RAIN[-20:].mean() / f09_nohacks_int_piControl_prec[-20:].mean()).round(2).values)
print('ARCTIC(P0): ', (100 * ARCTIC_int_piControl_RAIN[-20:].mean() / ARCTIC_int_piControl_prec[-20:].mean()).round(2).values)
print('')

print('F09M(P1): ', (100 * f09_int_1pctCO2_4xext_RAIN[130:150].mean() / f09_int_1pctCO2_4xext_prec[130:150].mean()).round(2).values)
print('F09(P1): ', (100 * f09_nohacks_int_1pctCO2_4xext_RAIN[130:150].mean() / f09_nohacks_int_1pctCO2_4xext_prec[130:150].mean()).round(2).values)
print('ARCTIC(P1): ', (100 * ARCTIC_int_1pctCO2_4xext_RAIN[130:150].mean() / ARCTIC_int_1pctCO2_4xext_prec[130:150].mean()).round(2).values)
print('')

print('F09M(P2): ', (100 * f09_int_1pctCO2_4xext_RAIN[230:250].mean() / f09_int_1pctCO2_4xext_prec[230:250].mean()).round(2).values)
print('F09(P2): ', (100 * f09_nohacks_int_1pctCO2_4xext_RAIN[230:250].mean() / f09_nohacks_int_1pctCO2_4xext_prec[230:250].mean()).round(2).values)
print('ARCTIC(P2): ', (100 * ARCTIC_int_1pctCO2_4xext_RAIN[230:250].mean() / ARCTIC_int_1pctCO2_4xext_prec[230:250].mean()).round(2).values)
print('')

print('F09M(P3): ', (100 * f09_int_1pctCO2_4xext_RAIN[330:350].mean() / f09_int_1pctCO2_4xext_prec[330:350].mean()).round(2).values)
print('F09(P3): ', (100 * f09_nohacks_int_1pctCO2_4xext_RAIN[330:350].mean() / f09_nohacks_int_1pctCO2_4xext_prec[330:350].mean()).round(2).values)
print('ARCTIC(P3): ', (100 * ARCTIC_int_1pctCO2_4xext_RAIN[330:350].mean() / ARCTIC_int_1pctCO2_4xext_prec[330:350].mean()).round(2).values)

Rain(%)

F09M(P0):  9.76
F09(P0):  8.98
ARCTIC(P0):  8.95

F09M(P1):  25.35
F09(P1):  24.48
ARCTIC(P1):  22.51

F09M(P2):  36.76
F09(P2):  32.21
ARCTIC(P2):  28.49

F09M(P3):  39.87
F09(P3):  37.36
ARCTIC(P3):  32.85


In [17]:
# Melt
print('Melt')
print('')

print('F09M(P0): ', BG7_int_melt[129:149].mean().round(2).values)
print('F09(P0): ', f09_nohacks_int_piControl_melt[-20:].mean().round(2).values)
print('ARCTIC(P0): ', ARCTIC_int_piControl_melt[-20:].mean().round(2).values)
print('')

print('F09M(P1): ', f09_int_1pctCO2_4xext_melt[130:150].mean().round(2).values)
print('F09(P1): ', f09_nohacks_int_1pctCO2_4xext_melt[130:150].mean().round(2).values)
print('ARCTIC(P1): ', ARCTIC_int_1pctCO2_4xext_melt[130:150].mean().round(2).values)
print('')

print('F09M(P2): ', f09_int_1pctCO2_4xext_melt[230:250].mean().round(2).values)
print('F09(P2): ', f09_nohacks_int_1pctCO2_4xext_melt[230:250].mean().round(2).values)
print('ARCTIC(P2): ', ARCTIC_int_1pctCO2_4xext_melt[230:250].mean().round(2).values)
print('')

print('F09M(P3): ', f09_int_1pctCO2_4xext_melt[330:350].mean().round(2).values)
print('F09(P3): ', f09_nohacks_int_1pctCO2_4xext_melt[330:350].mean().round(2).values)
print('ARCTIC(P3): ', ARCTIC_int_1pctCO2_4xext_melt[330:350].mean().round(2).values)

Melt

F09M(P0):  485.41
F09(P0):  475.83
ARCTIC(P0):  467.79

F09M(P1):  2146.85
F09(P1):  2399.91
ARCTIC(P1):  1986.44

F09M(P2):  3662.01
F09(P2):  3605.58
ARCTIC(P2):  3082.71

F09M(P3):  4008.67
F09(P3):  4073.05
ARCTIC(P3):  3805.64


In [32]:
# Sublimation
print('Sublimation')
print('')

print('F09M(P0): ', )
print('F09(P0): ', f09_nohacks_int_piControl_QFLX_EVAP_TOT[-20:].mean().round(2).values)
print('ARCTIC(P0): ', ARCTIC_int_piControl_p4_QFLX_EVAP_TOT[-20:].mean().round(2).values)
print('')

print('F09M(P1): ', f09_int_1pctCO2_4xext_QFLX_EVAP_TOT[130:150].mean().round(2).values)
print('F09(P1): ', f09_nohacks_int_1pctCO2_4xext_QFLX_EVAP_TOT[130:150].mean().round(2).values)
print('ARCTIC(P1): ', ARCTIC_int_1pctCO2_4xext_QFLX_EVAP_TOT[130:150].mean().round(2).values)
print('')

print('F09M(P2): ', f09_int_1pctCO2_4xext_QFLX_EVAP_TOT[230:250].mean().round(2).values)
print('F09(P2): ', f09_nohacks_int_1pctCO2_4xext_QFLX_EVAP_TOT[230:250].mean().round(2).values)
print('ARCTIC(P2): ', ARCTIC_int_1pctCO2_4xext_QFLX_EVAP_TOT[230:250].mean().round(2).values)
print('')

print('F09M(P3): ', f09_int_1pctCO2_4xext_QFLX_EVAP_TOT[330:350].mean().round(2).values)
print('F09(P3): ', f09_nohacks_int_1pctCO2_4xext_QFLX_EVAP_TOT[330:350].mean().round(2).values)
print('ARCTIC(P3): ', ARCTIC_int_1pctCO2_4xext_QFLX_EVAP_TOT[330:350].mean().round(2).values)

Sublimation

F09M(P0): 
F09(P0):  51.62
ARCTIC(P0):  57.39

F09M(P1):  59.61
F09(P1):  53.21
ARCTIC(P1):  70.29

F09M(P2):  34.32
F09(P2):  33.72
ARCTIC(P2):  51.71

F09M(P3):  18.99
F09(P3):  18.79
ARCTIC(P3):  32.3


In [33]:
# Refreezing
print('Refreezing')
print('')

print('F09M(P0): ', (BG7_int_melt[129:149].mean() + BG7_int_RAIN[129:149].mean() - BG7_int_QRUNOFF[129:149].mean()).round(2).values)
print('F09(P0): ', (f09_nohacks_int_piControl_melt[-20:].mean() + f09_nohacks_int_piControl_RAIN[-20:].mean() - f09_nohacks_int_piControl_QRUNOFF[-20:].mean()).round(2).values)
print('ARCTIC(P0): ', (ARCTIC_int_piControl_melt[-20:].mean() + ARCTIC_int_piControl_RAIN[-20:].mean() - ARCTIC_int_piControl_p4_QRUNOFF[-20:].mean()).round(2).values)
print('')

print('F09M(P1): ', (f09_int_1pctCO2_4xext_melt[130:150].mean() + f09_int_1pctCO2_4xext_RAIN[130:150].mean() - f09_int_1pctCO2_4xext_QRUNOFF[130:150].mean()).round(2).values)
print('F09(P1): ', (f09_nohacks_int_1pctCO2_4xext_melt[130:150].mean() + f09_nohacks_int_1pctCO2_4xext_RAIN[130:150].mean() - f09_nohacks_int_1pctCO2_4xext_QRUNOFF[130:150].mean()).round(2).values)
print('ARCTIC(P1): ', (ARCTIC_int_1pctCO2_4xext_melt[130:150].mean() + ARCTIC_int_1pctCO2_4xext_RAIN[130:150].mean() - ARCTIC_int_1pctCO2_4xext_QRUNOFF[130:150].mean()).round(2).values)
print('')

print('F09M(P2): ', (f09_int_1pctCO2_4xext_melt[230:250].mean() + f09_int_1pctCO2_4xext_RAIN[230:250].mean() - f09_int_1pctCO2_4xext_QRUNOFF[230:250].mean()).round(2).values)
print('F09(P2): ', (f09_nohacks_int_1pctCO2_4xext_melt[230:250].mean() + f09_nohacks_int_1pctCO2_4xext_RAIN[230:250].mean() - f09_nohacks_int_1pctCO2_4xext_QRUNOFF[230:250].mean()).round(2).values)
print('ARCTIC(P2): ', (ARCTIC_int_1pctCO2_4xext_melt[230:250].mean() + ARCTIC_int_1pctCO2_4xext_RAIN[230:250].mean() - ARCTIC_int_1pctCO2_4xext_QRUNOFF[230:250].mean()).round(2).values)
print('')

print('F09M(P3): ', (f09_int_1pctCO2_4xext_melt[330:350].mean() + f09_int_1pctCO2_4xext_RAIN[330:350].mean() - f09_int_1pctCO2_4xext_QRUNOFF[330:350].mean()).round(2).values)
print('F09(P3): ', (f09_nohacks_int_1pctCO2_4xext_melt[330:350].mean() + f09_nohacks_int_1pctCO2_4xext_RAIN[330:350].mean() - f09_nohacks_int_1pctCO2_4xext_QRUNOFF[330:350].mean()).round(2).values)
print('ARCTIC(P3): ', (ARCTIC_int_1pctCO2_4xext_melt[330:350].mean() + ARCTIC_int_1pctCO2_4xext_RAIN[330:350].mean() - ARCTIC_int_1pctCO2_4xext_QRUNOFF[330:350].mean()).round(2).values)

Refreezing

F09M(P0):  141.8
F09(P0):  294.52
ARCTIC(P0):  306.71

F09M(P1):  679.8
F09(P1):  871.9
ARCTIC(P1):  757.93

F09M(P2):  784.37
F09(P2):  955.75
ARCTIC(P2):  823.99

F09M(P3):  780.62
F09(P3):  1018.65
ARCTIC(P3):  829.51


In [11]:
# Refreezing(%)
print('Refreezing(%)')
print('')

print('F09M(P0): ', (100 * (BG7_int_melt[129:149].mean() + BG7_int_RAIN[129:149].mean() - BG7_int_QRUNOFF[129:149].mean()) / (BG7_int_melt[129:149].mean() + BG7_int_RAIN[129:149].mean())).round(2).values)
print('F09(P0): ', (100 * (f09_nohacks_int_piControl_melt[-20:].mean() + f09_nohacks_int_piControl_RAIN[-20:].mean() - f09_nohacks_int_piControl_QRUNOFF[-20:].mean()) / (f09_nohacks_int_piControl_melt[-20:].mean() + f09_nohacks_int_piControl_RAIN[-20:].mean())).round(2).values)
print('ARCTIC(P0): ', (100 * (ARCTIC_int_piControl_melt[-20:].mean() + ARCTIC_int_piControl_RAIN[-20:].mean() - ARCTIC_int_piControl_p4_QRUNOFF[-20:].mean()) / (ARCTIC_int_piControl_melt[-20:].mean() + ARCTIC_int_piControl_RAIN[-20:].mean())).round(2).values)
print('')

print('F09M(P1): ', (100 * (f09_int_1pctCO2_4xext_melt[130:150].mean() + f09_int_1pctCO2_4xext_RAIN[130:150].mean() - f09_int_1pctCO2_4xext_QRUNOFF[130:150].mean()) / (f09_int_1pctCO2_4xext_melt[130:150].mean() + f09_int_1pctCO2_4xext_RAIN[130:150].mean())).round(2).values)
print('F09(P1): ', (100 * (f09_nohacks_int_1pctCO2_4xext_melt[130:150].mean() + f09_nohacks_int_1pctCO2_4xext_RAIN[130:150].mean() - f09_nohacks_int_1pctCO2_4xext_QRUNOFF[130:150].mean()) / (f09_nohacks_int_1pctCO2_4xext_melt[130:150].mean() + f09_nohacks_int_1pctCO2_4xext_RAIN[130:150].mean())).round(2).values)
print('ARCTIC(P1): ', (100 * (ARCTIC_int_1pctCO2_4xext_melt[130:150].mean() + ARCTIC_int_1pctCO2_4xext_RAIN[130:150].mean() - ARCTIC_int_1pctCO2_4xext_QRUNOFF[130:150].mean()) / (ARCTIC_int_1pctCO2_4xext_melt[130:150].mean() + ARCTIC_int_1pctCO2_4xext_RAIN[130:150].mean())).round(2).values)
print('')

print('F09M(P2): ', (100 * (f09_int_1pctCO2_4xext_melt[230:250].mean() + f09_int_1pctCO2_4xext_RAIN[230:250].mean() - f09_int_1pctCO2_4xext_QRUNOFF[230:250].mean()) / (f09_int_1pctCO2_4xext_melt[230:250].mean() + f09_int_1pctCO2_4xext_RAIN[230:250].mean())).round(2).values)
print('F09(P2): ', (100 * (f09_nohacks_int_1pctCO2_4xext_melt[230:250].mean() + f09_nohacks_int_1pctCO2_4xext_RAIN[230:250].mean() - f09_nohacks_int_1pctCO2_4xext_QRUNOFF[230:250].mean()) / (f09_nohacks_int_1pctCO2_4xext_melt[230:250].mean() + f09_nohacks_int_1pctCO2_4xext_RAIN[230:250].mean())).round(2).values)
print('ARCTIC(P2): ', (100 * (ARCTIC_int_1pctCO2_4xext_melt[230:250].mean() + ARCTIC_int_1pctCO2_4xext_RAIN[230:250].mean() - ARCTIC_int_1pctCO2_4xext_QRUNOFF[230:250].mean()) / (ARCTIC_int_1pctCO2_4xext_melt[230:250].mean() + ARCTIC_int_1pctCO2_4xext_RAIN[230:250].mean())).round(2).values)
print('')

print('F09M(P3): ', (100 * (f09_int_1pctCO2_4xext_melt[330:350].mean() + f09_int_1pctCO2_4xext_RAIN[330:350].mean() - f09_int_1pctCO2_4xext_QRUNOFF[330:350].mean()) / (f09_int_1pctCO2_4xext_melt[330:350].mean() + f09_int_1pctCO2_4xext_RAIN[330:350].mean())).round(2).values)
print('F09(P3): ', (100 * (f09_nohacks_int_1pctCO2_4xext_melt[330:350].mean() + f09_nohacks_int_1pctCO2_4xext_RAIN[330:350].mean() - f09_nohacks_int_1pctCO2_4xext_QRUNOFF[330:350].mean()) / (f09_nohacks_int_1pctCO2_4xext_melt[330:350].mean() + f09_nohacks_int_1pctCO2_4xext_RAIN[330:350].mean())).round(2).values)
print('ARCTIC(P3): ', (100 * (ARCTIC_int_1pctCO2_4xext_melt[330:350].mean() + ARCTIC_int_1pctCO2_4xext_RAIN[330:350].mean() - ARCTIC_int_1pctCO2_4xext_QRUNOFF[330:350].mean()) / (ARCTIC_int_1pctCO2_4xext_melt[330:350].mean() + ARCTIC_int_1pctCO2_4xext_RAIN[330:350].mean())).round(2).values)

Refreezing(%)

F09M(P0):  24.56
F09(P0):  51.86
ARCTIC(P0):  55.44

F09M(P1):  28.18
F09(P1):  32.16
ARCTIC(P1):  33.59

F09M(P2):  19.28
F09(P2):  23.61
ARCTIC(P2):  23.91

F09M(P3):  17.46
F09(P3):  22.33
ARCTIC(P3):  19.65


In [12]:
# SMB
print('SMB')
print('')

print('F09M(P0): ', )
print('F09(P0): ', (f09_nohacks_int_piControl_prec[-20:].mean() - f09_nohacks_int_piControl_QFLX_EVAP_TOT[-20:].mean() - f09_nohacks_int_piControl_QRUNOFF[-20:].mean()).round(2).values)
print('ARCTIC(P0): ', (ARCTIC_int_piControl_prec[-20:].mean() - ARCTIC_int_piControl_p4_QFLX_EVAP_TOT[-20:].mean() - ARCTIC_int_piControl_p4_QRUNOFF[-20:].mean()).round(2).values)
print('')

print('F09M(P1): ', (f09_int_1pctCO2_4xext_prec[130:150].mean() - f09_int_1pctCO2_4xext_QFLX_EVAP_TOT[130:150].mean() - f09_int_1pctCO2_4xext_QRUNOFF[130:150].mean()).round(2).values)
print('F09(P1): ', (f09_nohacks_int_1pctCO2_4xext_prec[130:150].mean() - f09_nohacks_int_1pctCO2_4xext_QFLX_EVAP_TOT[130:150].mean() - f09_nohacks_int_1pctCO2_4xext_QRUNOFF[130:150].mean()).round(2).values)
print('ARCTIC(P1): ', (ARCTIC_int_1pctCO2_4xext_prec[130:150].mean() - ARCTIC_int_1pctCO2_4xext_QFLX_EVAP_TOT[130:150].mean() - ARCTIC_int_1pctCO2_4xext_QRUNOFF[130:150].mean()).round(2).values)
print('')

print('F09M(P2): ', (f09_int_1pctCO2_4xext_prec[230:250].mean() - f09_int_1pctCO2_4xext_QFLX_EVAP_TOT[230:250].mean() - f09_int_1pctCO2_4xext_QRUNOFF[230:250].mean()).round(2).values)
print('F09(P2): ', (f09_nohacks_int_1pctCO2_4xext_prec[230:250].mean() - f09_nohacks_int_1pctCO2_4xext_QFLX_EVAP_TOT[230:250].mean() - f09_nohacks_int_1pctCO2_4xext_QRUNOFF[230:250].mean()).round(2).values)
print('ARCTIC(P2): ', (ARCTIC_int_1pctCO2_4xext_prec[230:250].mean() - ARCTIC_int_1pctCO2_4xext_QFLX_EVAP_TOT[230:250].mean() - ARCTIC_int_1pctCO2_4xext_QRUNOFF[230:250].mean()).round(2).values)
print('')

print('F09M(P3): ', (f09_int_1pctCO2_4xext_prec[330:350].mean() - f09_int_1pctCO2_4xext_QFLX_EVAP_TOT[330:350].mean() - f09_int_1pctCO2_4xext_QRUNOFF[330:350].mean()).round(2).values)
print('F09(P3): ', (f09_nohacks_int_1pctCO2_4xext_prec[330:350].mean() - f09_nohacks_int_1pctCO2_4xext_QFLX_EVAP_TOT[330:350].mean() - f09_nohacks_int_1pctCO2_4xext_QRUNOFF[330:350].mean()).round(2).values)
print('ARCTIC(P3): ', (ARCTIC_int_1pctCO2_4xext_prec[330:350].mean() - ARCTIC_int_1pctCO2_4xext_QFLX_EVAP_TOT[330:350].mean() - ARCTIC_int_1pctCO2_4xext_QRUNOFF[330:350].mean()).round(2).values)

SMB

F09M(P0): 
F09(P0):  701.28
ARCTIC(P0):  650.97

F09M(P1):  -744.89
F09(P1):  -619.93
ARCTIC(P1):  -369.16

F09M(P2):  -2212.78
F09(P2):  -1751.83
ARCTIC(P2):  -1397.5

F09M(P3):  -2551.78
F09(P3):  -2254.09
ARCTIC(P3):  -2159.02


## 2. SEB

MHF = SWnet + LWnet + SHF + LHF + GHF, 
where 

MHF = FSM(_ICE) + QICE_MELT*hfus
<br>
SWnet = FSDS - FSR(_ICE)
<br>
LWnet = (FLDS(_ICE) - FIRE(_ICE)) * -1 (this one is defined away from the surface, so you need to swap sign and therefore * -1)
<br>
SHF = FSH(_ICE) * -1 (same story here)
<br>
LHF = EFLX_LH_TOT(_ICE) *-1 (same story here) 
<br>
GHF = (FGR(_ICE) - MHF) *-1 (as mentioned above the sign need to be swapped here as well). 

### Read data

In [4]:
## BG iteration 7
BG7_TREFHT_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/BG7/BG7_TREFHT_JJA_yearly.nc')['TREFHT'][:]
BG7_TS_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/BG7/BG7_TS_JJA_yearly.nc')['TS'][:]
BG7_FSDS_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/BG7/BG7_FSDS_JJA_yearly.nc')['FSDS'][:]
BG7_FSA_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/BG7/BG7_FSA_JJA_yearly.nc')['FSA'][:]
BG7_FLDS_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/BG7/BG7_FLDS_JJA_yearly.nc')['FLDS'][:]
BG7_FIRA_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/BG7/BG7_FIRA_JJA_yearly.nc')['FIRA'][:]
BG7_FSH_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/BG7/BG7_FSH_JJA_yearly.nc')['FSH'][:]
BG7_EFLX_LH_TOT_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/BG7/BG7_EFLX_LH_TOT_JJA_yearly.nc')['EFLX_LH_TOT'][:]
BG7_FSM_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/BG7/BG7_FSM_JJA_yearly.nc')['FSM'][:]
BG7_FICE_MELT_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/BG7/BG7_QICE_MELT_JJA_yearly.nc')['QICE_MELT'][:] * 3.34e5 #[W/m2]
#BG7_FGR_JJA = xr.open_dataset(home_dir+'dynamic_GrIS/BG7/BG7_FGR_JJA_yearly.nc')['FGR'][:] # Var not saved, can calculate

In [5]:
## f09muntjewerf
data_dir = '/glade/derecho/scratch/ziqiyin/2022-VR-dynamic-GrIS/'

# piControl
f09_piControl_FSDS_JJA = xr.open_dataset(data_dir+'f09_FSDS_piControl_JJA_yearly.nc')['FSDS'][:]
f09_piControl_FSA_JJA = xr.open_dataset(data_dir+'f09_FSA_piControl_JJA_yearly.nc')['FSA'][:]
f09_piControl_FLDS_JJA = xr.open_dataset(data_dir+'f09_FLDS_piControl_JJA_yearly.nc')['FLDS'][:]
f09_piControl_FIRA_JJA = xr.open_dataset(data_dir+'f09_FIRA_piControl_JJA_yearly.nc')['FIRA'][:]
f09_piControl_FSH_JJA = xr.open_dataset(data_dir+'f09_FSH_piControl_JJA_yearly.nc')['FSH'][:]
f09_piControl_EFLX_LH_TOT_JJA = xr.open_dataset(data_dir+'f09_EFLX_LH_TOT_piControl_JJA_yearly.nc')['EFLX_LH_TOT'][:]
f09_piControl_FSM_JJA = xr.open_dataset(data_dir+'f09_FSM_piControl_JJA_yearly.nc')['FSM'][:]
f09_piControl_FICE_MELT_JJA = xr.open_dataset(data_dir+'f09_QICE_MELT_piControl_JJA_yearly.nc')['QICE_MELT'][:] * 3.34e5 #[W/m2]
f09_piControl_FGR_JJA = xr.open_dataset(data_dir+'f09_FGR_piControl_JJA_yearly.nc')['FGR'][:]

# 1pctCO2
f09_1pctCO2_TREFHT_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_TREFHT_1pctCO2_JJA_yearly.nc')['TREFHT'][:140]
f09_1pctCO2_TS_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_TS_1pctCO2_JJA_yearly.nc')['TS'][:140]
f09_1pctCO2_FSDS_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_FSDS_1pctCO2_JJA.nc')['FSDS'][:140]
f09_1pctCO2_FSA_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_FSA_1pctCO2_JJA.nc')['FSA'][:140]
f09_1pctCO2_FLDS_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_FLDS_1pctCO2_JJA.nc')['FLDS'][:140]
f09_1pctCO2_FIRA_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_FIRA_1pctCO2_JJA.nc')['FIRA'][:140]
f09_1pctCO2_FSH_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_FSH_1pctCO2_JJA.nc')['FSH'][:140]
f09_1pctCO2_EFLX_LH_TOT_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_EFLX_LH_TOT_1pctCO2_JJA.nc')['EFLX_LH_TOT'][:140]
f09_1pctCO2_FSM_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_FSM_1pctCO2_JJA.nc')['FSM'][:140]
f09_1pctCO2_FICE_MELT_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_QICE_MELT_1pctCO2_JJA.nc')['QICE_MELT'][:140] * 3.34e5 #[W/m2]
f09_1pctCO2_FGR_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_FGR_1pctCO2_JJA.nc')['FGR'][:140]
f09_1pctCO2_PHIS = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_PHIS_1pctCO2_JJA_yearly.nc')['PHIS'][:140]

# 4xCO2
f09_4xext_TREFHT_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_TREFHT_4xext_JJA_yearly.nc')['TREFHT'][:]
f09_4xext_TS_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_TS_4xext_JJA_yearly.nc')['TS'][:]
f09_4xext_FSDS_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_FSDS_4xext_JJA.nc')['FSDS'][:]
f09_4xext_FSA_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_FSA_4xext_JJA.nc')['FSA'][:]
f09_4xext_FLDS_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_FLDS_4xext_JJA.nc')['FLDS'][:]
f09_4xext_FIRA_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_FIRA_4xext_JJA.nc')['FIRA'][:]
f09_4xext_FSH_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_FSH_4xext_JJA.nc')['FSH'][:]
f09_4xext_EFLX_LH_TOT_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_EFLX_LH_TOT_4xext_JJA.nc')['EFLX_LH_TOT'][:]
f09_4xext_FSM_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_FSM_4xext_JJA.nc')['FSM'][:]
f09_4xext_FICE_MELT_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_QICE_MELT_4xext_JJA.nc')['QICE_MELT'][:] * 3.34e5 #[W/m2]
f09_4xext_FGR_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_FGR_4xext_JJA.nc')['FGR'][:]
f09_4xext_PHIS = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_PHIS_4xext_JJA_yearly.nc')['PHIS'][:]

In [6]:
## f09nohacks
# piControl
f09_nohacks_piControl_TREFHT_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_TREFHT_piControl_JJA_yearly.nc')['TREFHT'][:]
f09_nohacks_piControl_TS_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_TS_piControl_JJA_yearly.nc')['TS'][:]
f09_nohacks_piControl_FSDS_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_FSDS_piControl_JJA_yearly.nc')['FSDS'][:]
f09_nohacks_piControl_FSR_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_FSR_piControl_JJA_yearly.nc')['FSR'][:]
f09_nohacks_piControl_FSA_JJA = f09_nohacks_piControl_FSDS_JJA - f09_nohacks_piControl_FSR_JJA
f09_nohacks_piControl_FLDS_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_FLDS_piControl_JJA_yearly.nc')['FLDS'][:]
f09_nohacks_piControl_FIRA_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_FIRA_piControl_JJA_yearly.nc')['FIRA'][:]
f09_nohacks_piControl_FSH_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_FSH_piControl_JJA_yearly.nc')['FSH'][:]
f09_nohacks_piControl_EFLX_LH_TOT_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_EFLX_LH_TOT_piControl_JJA_yearly.nc')['EFLX_LH_TOT'][:]
f09_nohacks_piControl_FSM_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_FSM_piControl_JJA_yearly.nc')['FSM'][:]
f09_nohacks_piControl_FICE_MELT_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_QICE_MELT_piControl_JJA_yearly.nc')['QICE_MELT'][:] * 3.34e5 #[W/m2]
f09_nohacks_piControl_FGR_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_FGR_piControl_JJA_yearly.nc')['FGR'][:]

# 1pctCO2
f09_nohacks_1pctCO2_TREFHT_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_TREFHT_1pctCO2_JJA_yearly.nc')['TREFHT'][:]
f09_nohacks_1pctCO2_TS_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_TS_1pctCO2_JJA_yearly.nc')['TS'][:]
f09_nohacks_1pctCO2_FSDS_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_FSDS_1pctCO2_JJA_yearly.nc')['FSDS'][:]
f09_nohacks_1pctCO2_FSR_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_FSR_1pctCO2_JJA_yearly.nc')['FSR'][:]
f09_nohacks_1pctCO2_FSA_JJA = f09_nohacks_1pctCO2_FSDS_JJA - f09_nohacks_1pctCO2_FSR_JJA
f09_nohacks_1pctCO2_FLDS_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_FLDS_1pctCO2_JJA_yearly.nc')['FLDS'][:]
f09_nohacks_1pctCO2_FIRA_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_FIRA_1pctCO2_JJA_yearly.nc')['FIRA'][:]
f09_nohacks_1pctCO2_FSH_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_FSH_1pctCO2_JJA_yearly.nc')['FSH'][:]
f09_nohacks_1pctCO2_EFLX_LH_TOT_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_EFLX_LH_TOT_1pctCO2_JJA_yearly.nc')['EFLX_LH_TOT'][:]
f09_nohacks_1pctCO2_FSM_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_FSM_1pctCO2_JJA_yearly.nc')['FSM'][:]
f09_nohacks_1pctCO2_FICE_MELT_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_QICE_MELT_1pctCO2_JJA_yearly.nc')['QICE_MELT'][:] * 3.34e5 #[W/m2]
f09_nohacks_1pctCO2_FGR_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_FGR_1pctCO2_JJA_yearly.nc')['FGR'][:]
f09_nohacks_1pctCO2_PHIS = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_PHIS_1pctCO2_JJA_yearly.nc')['PHIS'][:]

# 4xext
f09_nohacks_4xext_TREFHT_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_TREFHT_4xext_JJA_yearly.nc')['TREFHT'][:]
f09_nohacks_4xext_TS_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_TS_4xext_JJA_yearly.nc')['TS'][:]
f09_nohacks_4xext_FSDS_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_FSDS_4xext_JJA_yearly.nc')['FSDS'][:]
f09_nohacks_4xext_FSR_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_FSR_4xext_JJA_yearly.nc')['FSR'][:]
f09_nohacks_4xext_FSA_JJA = f09_nohacks_4xext_FSDS_JJA - f09_nohacks_4xext_FSR_JJA
f09_nohacks_4xext_FLDS_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_FLDS_4xext_JJA_yearly.nc')['FLDS'][:]
f09_nohacks_4xext_FIRA_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_FIRA_4xext_JJA_yearly.nc')['FIRA'][:]
f09_nohacks_4xext_FSH_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_FSH_4xext_JJA_yearly.nc')['FSH'][:]
f09_nohacks_4xext_EFLX_LH_TOT_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_EFLX_LH_TOT_4xext_JJA_yearly.nc')['EFLX_LH_TOT'][:]
f09_nohacks_4xext_FSM_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_FSM_4xext_JJA_yearly.nc')['FSM'][:]
f09_nohacks_4xext_FICE_MELT_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_QICE_MELT_4xext_JJA_yearly.nc')['QICE_MELT'][:] * 3.34e5 #[W/m2]
f09_nohacks_4xext_FGR_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_FGR_4xext_JJA_yearly.nc')['FGR'][:]
f09_nohacks_4xext_PHIS = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_PHIS_4xext_JJA_yearly.nc')['PHIS'][:]

In [7]:
## Arctic grid
# piControl
ARCTIC_piControl_p4_TREFHT_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_TREFHT_piControl_p4_JJA_yearly.nc')['TREFHT'][:]
ARCTIC_piControl_p4_TS_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_TS_piControl_p4_JJA_yearly.nc')['TS'][:]
ARCTIC_piControl_p4_FSDS_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_FSDS_piControl_p4_JJA.nc')['FSDS'][:]
ARCTIC_piControl_p4_FSA_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_FSA_piControl_p4_JJA.nc')['FSA'][:]
ARCTIC_piControl_p4_FLDS_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_FLDS_piControl_p4_JJA.nc')['FLDS'][:]
ARCTIC_piControl_p4_FIRA_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_FIRA_piControl_p4_JJA.nc')['FIRA'][:]
ARCTIC_piControl_p4_FSH_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_FSH_piControl_p4_JJA.nc')['FSH'][:]
ARCTIC_piControl_p4_EFLX_LH_TOT_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_EFLX_LH_TOT_piControl_p4_JJA.nc')['EFLX_LH_TOT'][:]
ARCTIC_piControl_p4_FSM_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_FSM_piControl_p4_JJA.nc')['FSM'][:]
ARCTIC_piControl_p4_FICE_MELT_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_QICE_MELT_piControl_p4_JJA.nc')['QICE_MELT'][:] * 3.34e5 #[W/m2]
ARCTIC_piControl_p4_FGR_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_FGR_piControl_p4_JJA.nc')['FGR'][:]

# 1pctCO2
ARCTIC_1pctCO2_TREFHT_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_TREFHT_1pctCO2_JJA_yearly.nc')['TREFHT'][:]
ARCTIC_1pctCO2_TS_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_TS_1pctCO2_JJA_yearly.nc')['TS'][:]
ARCTIC_1pctCO2_FSDS_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_FSDS_1pctCO2_JJA.nc')['FSDS'][:]
ARCTIC_1pctCO2_FSA_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_FSA_1pctCO2_JJA.nc')['FSA'][:]
ARCTIC_1pctCO2_FLDS_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_FLDS_1pctCO2_JJA.nc')['FLDS'][:]
ARCTIC_1pctCO2_FIRA_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_FIRA_1pctCO2_JJA.nc')['FIRA'][:]
ARCTIC_1pctCO2_FSH_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_FSH_1pctCO2_JJA.nc')['FSH'][:]
ARCTIC_1pctCO2_EFLX_LH_TOT_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_EFLX_LH_TOT_1pctCO2_JJA.nc')['EFLX_LH_TOT'][:]
ARCTIC_1pctCO2_FSM_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_FSM_1pctCO2_JJA.nc')['FSM'][:]
ARCTIC_1pctCO2_FICE_MELT_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_QICE_MELT_1pctCO2_JJA.nc')['QICE_MELT'][:] * 3.34e5 #[W/m2]
ARCTIC_1pctCO2_FGR_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_FGR_1pctCO2_JJA.nc')['FGR'][:]
ARCTIC_1pctCO2_PHIS = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_PHIS_1pctCO2_JJA_yearly.nc')['PHIS'][:]

# 4xext
ARCTIC_4xext_TREFHT_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_TREFHT_4xext_JJA_yearly.nc')['TREFHT'][:]
ARCTIC_4xext_TS_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_TS_4xext_JJA_yearly.nc')['TS'][:]
ARCTIC_4xext_FSDS_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_FSDS_4xext_JJA.nc')['FSDS'][:]
ARCTIC_4xext_FSA_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_FSA_4xext_JJA.nc')['FSA'][:]
ARCTIC_4xext_FLDS_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_FLDS_4xext_JJA.nc')['FLDS'][:]
ARCTIC_4xext_FIRA_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_FIRA_4xext_JJA.nc')['FIRA'][:]
ARCTIC_4xext_FSH_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_FSH_4xext_JJA.nc')['FSH'][:]
ARCTIC_4xext_EFLX_LH_TOT_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_EFLX_LH_TOT_4xext_JJA.nc')['EFLX_LH_TOT'][:]
ARCTIC_4xext_FSM_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_FSM_4xext_JJA.nc')['FSM'][:]
ARCTIC_4xext_FICE_MELT_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_QICE_MELT_4xext_JJA.nc')['QICE_MELT'][:] * 3.34e5 #[W/m2]
ARCTIC_4xext_FGR_JJA = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_FGR_4xext_JJA.nc')['FGR'][:]
ARCTIC_4xext_PHIS = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_PHIS_4xext_JJA_yearly.nc')['PHIS'][:]

In [8]:
## Area and land fraction
Re = 6371.22 * 1e3 #[m]

f09_mask_file1 = '/glade/p/cesmdata/cseg/inputdata/share/domains/domain.lnd.fv0.9x1.25_gx1v7.151020.nc'
f09_area = xr.open_dataset(f09_mask_file1)['area'] #[Radian2]
f09_area = f09_area.values * Re**2

f09_mask_file2 = '/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850G.f09_g17_gl4.CMIP6-1pctCO2to4x-withism.001/lnd/proc/tseries/month_1/b.e21.B1850G.f09_g17_gl4.CMIP6-1pctCO2to4x-withism.001.clm2.h0.RAIN.000101-005012.nc'
f09_land_frac = xr.open_dataset(f09_mask_file2)['landfrac'] #fraction of grid cell that is active

ARCTIC_mask_file1 = '/glade/p/cesmdata/cseg/inputdata/share/domains/domain.lnd.ne0np4.ARCTIC.ne30x4_tx0.1v2.200626.nc'
ARCTIC_area = xr.open_dataset(ARCTIC_mask_file1)['area'] #[Radian2]
ARCTIC_area = ARCTIC_area.values * Re**2

ARCTIC_mask_file2 = '/glade/campaign/cesm/development/amwg/aherring/cesm2_2_alpha06d.poparctic_B1850G1PCT_ne0ARCTICne30x4_g17_8192pes_220705_Nx2yrs.001/lnd/hist/cesm2_2_alpha06d.poparctic_B1850G1PCT_ne0ARCTICne30x4_g17_8192pes_220705_Nx2yrs.001.clm2.h0.0001-01.nc' #directly from clm2.h0 files
ARCTIC_land_frac = xr.open_dataset(ARCTIC_mask_file2)['landfrac']

In [9]:
## Mask
# f09
# Generate Greenland mask
#f09_mask_file1 = '/glade/work/aherring/grids/uniform-res/f09/clm_surfdata_5_0/surfdata_f09_hist_78pfts_CMIP6_simyr1979_c200430.nc'
#f09_mask_file1 = '/glade/work/aherring/grids/uniform-res/f09/clm_surfdata_5_0/surfdata_0.9x1.25_hist_78pfts_CMIP5_simyr1850_c200226.nc'
f09_mask_file1 = '/glade/work/aherring/grids/uniform-res/f09/clm_surfdata_5_0/surfdata_f09_hist_78pfts_CMIP6_simyr1850_c200429.nc'
f09_GrIS_mask0 = xr.open_dataset(f09_mask_file1)['GLACIER_REGION']
f09_GrIS_mask0 = xr.where(f09_GrIS_mask0==2, 1, np.nan)
# Glacier fraction of land area
f09_glacier_frac_piControl = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_PCT_LANDUNIT_piControl_yearly.nc')['PCT_LANDUNIT'][:,3,:,:]
f09_mask_file2 = '/glade/campaign/cgd/amp/aherring/archive/cam6_2_022.se_FHIST_f09_f09_mg17_1800pes_200507_mg3-Nx5yrs/lnd/hist/cam6_2_022.se_FHIST_f09_f09_mg17_1800pes_200507_mg3-Nx5yrs.clm2.h0.1979-01.nc'
f09_glacier_frac_const = xr.open_dataset(f09_mask_file2)['PCT_LANDUNIT'][0,3,:,:] #ltype_landice_multiple_elevation_classes
f09_glacier_frac_1pctCO2 = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_PCT_LANDUNIT_1pctCO2_yearly.nc')['PCT_LANDUNIT'][:140,3,:,:]
f09_glacier_frac_4xext = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09muntjewerf/f09_PCT_LANDUNIT_4xext_yearly.nc')['PCT_LANDUNIT'][:,3,:,:]
f09_glacier_frac = xr.concat([f09_glacier_frac_1pctCO2[:140], f09_glacier_frac_4xext], dim='time')
# Land fraction of grid cell
#f09_mask_file3 = '/glade/p/cesmdata/cseg/inputdata/share/domains/domain.lnd.fv0.9x1.25_gx1v7.151020.nc'
f09_mask_file3 = '/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850G.f09_g17_gl4.CMIP6-1pctCO2to4x-withism.001/lnd/proc/tseries/month_1/b.e21.B1850G.f09_g17_gl4.CMIP6-1pctCO2to4x-withism.001.clm2.h0.RAIN.000101-005012.nc'
f09_land_frac = xr.open_dataset(f09_mask_file3)['landfrac'] #fraction of grid cell that is active
f09_GrIS_mask_piControl = f09_GrIS_mask0.values * f09_glacier_frac_piControl.values * f09_land_frac.values / 100
f09_GrIS_mask_const = f09_GrIS_mask0.values * f09_glacier_frac_const.values * f09_land_frac.values / 100
f09_GrIS_mask = f09_GrIS_mask0.values * f09_glacier_frac.values * f09_land_frac.values / 100

# f09nohacks
f09_nohacks_glacier_frac_piControl = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_PCT_LANDUNIT_piControl_yearly.nc')['PCT_LANDUNIT'][:,3,:,:]
f09_nohacks_GrIS_mask_piControl = f09_GrIS_mask0.values * f09_nohacks_glacier_frac_piControl.values * f09_land_frac.values / 100
f09_nohacks_glacier_frac_1pctCO2 = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_PCT_LANDUNIT_1pctCO2_yearly.nc')['PCT_LANDUNIT'][:,3,:,:]
f09_nohacks_glacier_frac_4xext = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/f09nohacks/f09_nohacks_PCT_LANDUNIT_4xext_yearly.nc')['PCT_LANDUNIT'][:,3,:,:]
f09_nohacks_glacier_frac = xr.concat([f09_nohacks_glacier_frac_1pctCO2, f09_nohacks_glacier_frac_4xext], dim='time')
f09_nohacks_GrIS_mask = f09_GrIS_mask0.values * f09_nohacks_glacier_frac.values * f09_land_frac.values / 100

# BG7
BG7_glacier_frac = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/BG7/BG7_PCT_LANDUNIT_yearly.nc')['PCT_LANDUNIT'][:,3,:,:]
BG7_GrIS_mask = f09_GrIS_mask0.values * BG7_glacier_frac.values * f09_land_frac.values / 100

# ARCTIC
# Generate Greenland mask
#ARCTIC_mask_file1 = '/glade/work/aherring/grids/var-res/ne0np4.ARCTIC.ne30x4/clm_surfdata_5_0/surfdata_ne0np4.ARCTIC.ne30x4_hist_78pfts_CMIP6_simyr2000_c200426.nc' #get same result
ARCTIC_mask_file1 = '/glade/work/aherring/grids/var-res/ne0np4.ARCTIC.ne30x4/clm_surfdata_5_0/surfdata_ne0np4.ARCTIC.ne30x4_hist_78pfts_CMIP6_simyr1850_c200426.nc'
ARCTIC_GrIS_mask0 = xr.open_dataset(ARCTIC_mask_file1)['GLACIER_REGION']
ARCTIC_GrIS_mask0 = xr.where(ARCTIC_GrIS_mask0==2, 1, np.nan)
# Glacier fraction of land area
ARCTIC_glacier_frac_piControl_p1 = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_PCT_LANDUNIT_piControl_p1_yearly.nc')['PCT_LANDUNIT'][:,3,:]
ARCTIC_glacier_frac_piControl_p2 = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_PCT_LANDUNIT_piControl_p2_yearly.nc')['PCT_LANDUNIT'][:,3,:]
ARCTIC_glacier_frac_piControl_p3 = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_PCT_LANDUNIT_piControl_p3_yearly.nc')['PCT_LANDUNIT'][:,3,:]
ARCTIC_glacier_frac_piControl_p4 = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_PCT_LANDUNIT_piControl_p4_yearly.nc')['PCT_LANDUNIT'][:,3,:]
ARCTIC_glacier_frac_piControl = xr.concat([ARCTIC_glacier_frac_piControl_p1, ARCTIC_glacier_frac_piControl_p2, ARCTIC_glacier_frac_piControl_p3, ARCTIC_glacier_frac_piControl_p4], dim='time')
ARCTIC_mask_file2 = '/glade/campaign/cgd/amp/aherring/archive/cam6_2_022.se_FHIST_ne0np4.ARCTIC.ne30x4_mt12_7680pes_200507_mg3-Nx2yrs/lnd/hist/cam6_2_022.se_FHIST_ne0np4.ARCTIC.ne30x4_mt12_7680pes_200507_mg3-Nx2yrs.clm2.h0.1979-01.nc'
ARCTIC_glacier_frac_const = xr.open_dataset(ARCTIC_mask_file2)['PCT_LANDUNIT'][0,3,:]
ARCTIC_glacier_frac_1pctCO2 = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_PCT_LANDUNIT_1pctCO2_yearly.nc')['PCT_LANDUNIT'][:,3,:]
ARCTIC_glacier_frac_4xext = xr.open_dataset(home_dir+'2022-VR-dynamic-GrIS/arctic/ARCTIC_PCT_LANDUNIT_4xext_yearly.nc')['PCT_LANDUNIT'][:,3,:]
ARCTIC_glacier_frac = xr.concat([ARCTIC_glacier_frac_1pctCO2, ARCTIC_glacier_frac_4xext], dim='time')
# Land fraction of grid cell
#ARCTIC_mask_file3 = '/glade/p/cesmdata/cseg/inputdata/share/domains/domain.lnd.ne0np4.ARCTIC.ne30x4_tx0.1v2.200626.nc' #Wrong domain, it is for B case
#ARCTIC_mask_file3 = '/glade/work/aherring/grids/var-res/ne0np4.ARCTIC.ne30x4/domains/domain.lnd.ne0np4.ARCTIC.ne30x4_gx1v7.200817.nc' #domain file with ['frac']
ARCTIC_mask_file3 = '/glade/campaign/cesm/development/amwg/aherring/cesm2_2_alpha06d.poparctic_B1850G1PCT_ne0ARCTICne30x4_g17_8192pes_220705_Nx2yrs.001/lnd/hist/cesm2_2_alpha06d.poparctic_B1850G1PCT_ne0ARCTICne30x4_g17_8192pes_220705_Nx2yrs.001.clm2.h0.0001-01.nc' #directly from clm2.h0 files
ARCTIC_land_frac = xr.open_dataset(ARCTIC_mask_file3)['landfrac']
ARCTIC_GrIS_mask_piControl = ARCTIC_GrIS_mask0.values * ARCTIC_glacier_frac_piControl.values * ARCTIC_land_frac.values / 100
ARCTIC_GrIS_mask_const = ARCTIC_GrIS_mask0.values * ARCTIC_glacier_frac_const.values * ARCTIC_land_frac.values / 100
ARCTIC_GrIS_mask = ARCTIC_GrIS_mask0.values * ARCTIC_glacier_frac.values * ARCTIC_land_frac.values / 100
#ARCTIC_GrIS_mask_v2 = ARCTIC_GrIS_mask0.values * ARCTIC_glacier_frac.values / 100

In [10]:
## Concat
f09_1pctCO2_4xext_TREFHT_JJA = xr.concat([f09_1pctCO2_TREFHT_JJA, f09_4xext_TREFHT_JJA], dim='time')
f09_1pctCO2_4xext_TS_JJA = xr.concat([f09_1pctCO2_TS_JJA, f09_4xext_TS_JJA], dim='time')
f09_1pctCO2_4xext_FSDS_JJA = xr.concat([f09_1pctCO2_FSDS_JJA, f09_4xext_FSDS_JJA], dim='time')
f09_1pctCO2_4xext_FSA_JJA = xr.concat([f09_1pctCO2_FSA_JJA, f09_4xext_FSA_JJA], dim='time')
f09_1pctCO2_4xext_FLDS_JJA = xr.concat([f09_1pctCO2_FLDS_JJA, f09_4xext_FLDS_JJA], dim='time')
f09_1pctCO2_4xext_FIRA_JJA = xr.concat([f09_1pctCO2_FIRA_JJA, f09_4xext_FIRA_JJA], dim='time')
f09_1pctCO2_4xext_FSH_JJA = xr.concat([f09_1pctCO2_FSH_JJA, f09_4xext_FSH_JJA], dim='time')
f09_1pctCO2_4xext_EFLX_LH_TOT_JJA = xr.concat([f09_1pctCO2_EFLX_LH_TOT_JJA, f09_4xext_EFLX_LH_TOT_JJA], dim='time')
f09_1pctCO2_4xext_FSM_JJA = xr.concat([f09_1pctCO2_FSM_JJA, f09_4xext_FSM_JJA], dim='time')
f09_1pctCO2_4xext_FICE_MELT_JJA = xr.concat([f09_1pctCO2_FICE_MELT_JJA, f09_4xext_FICE_MELT_JJA], dim='time')
f09_1pctCO2_4xext_FGR_JJA = xr.concat([f09_1pctCO2_FGR_JJA, f09_4xext_FGR_JJA], dim='time')
f09_1pctCO2_4xext_PHIS = xr.concat([f09_1pctCO2_PHIS, f09_4xext_PHIS], dim='time')

f09_nohacks_1pctCO2_4xext_TREFHT_JJA = xr.concat([f09_nohacks_1pctCO2_TREFHT_JJA, f09_nohacks_4xext_TREFHT_JJA], dim='time')
f09_nohacks_1pctCO2_4xext_TS_JJA = xr.concat([f09_nohacks_1pctCO2_TS_JJA, f09_nohacks_4xext_TS_JJA], dim='time')
f09_nohacks_1pctCO2_4xext_FSDS_JJA = xr.concat([f09_nohacks_1pctCO2_FSDS_JJA, f09_nohacks_4xext_FSDS_JJA], dim='time')
f09_nohacks_1pctCO2_4xext_FSA_JJA = xr.concat([f09_nohacks_1pctCO2_FSA_JJA, f09_nohacks_4xext_FSA_JJA], dim='time')
f09_nohacks_1pctCO2_4xext_FLDS_JJA = xr.concat([f09_nohacks_1pctCO2_FLDS_JJA, f09_nohacks_4xext_FLDS_JJA], dim='time')
f09_nohacks_1pctCO2_4xext_FIRA_JJA = xr.concat([f09_nohacks_1pctCO2_FIRA_JJA, f09_nohacks_4xext_FIRA_JJA], dim='time')
f09_nohacks_1pctCO2_4xext_FSH_JJA = xr.concat([f09_nohacks_1pctCO2_FSH_JJA, f09_nohacks_4xext_FSH_JJA], dim='time')
f09_nohacks_1pctCO2_4xext_EFLX_LH_TOT_JJA = xr.concat([f09_nohacks_1pctCO2_EFLX_LH_TOT_JJA, f09_nohacks_4xext_EFLX_LH_TOT_JJA], dim='time')
f09_nohacks_1pctCO2_4xext_FSM_JJA = xr.concat([f09_nohacks_1pctCO2_FSM_JJA, f09_nohacks_4xext_FSM_JJA], dim='time')
f09_nohacks_1pctCO2_4xext_FICE_MELT_JJA = xr.concat([f09_nohacks_1pctCO2_FICE_MELT_JJA, f09_nohacks_4xext_FICE_MELT_JJA], dim='time')
f09_nohacks_1pctCO2_4xext_FGR_JJA = xr.concat([f09_nohacks_1pctCO2_FGR_JJA, f09_nohacks_4xext_FGR_JJA], dim='time')
f09_nohacks_1pctCO2_4xext_PHIS = xr.concat([f09_nohacks_1pctCO2_PHIS, f09_nohacks_4xext_PHIS], dim='time')

ARCTIC_1pctCO2_4xext_TREFHT_JJA = xr.concat([ARCTIC_1pctCO2_TREFHT_JJA, ARCTIC_4xext_TREFHT_JJA], dim='time')
ARCTIC_1pctCO2_4xext_TS_JJA = xr.concat([ARCTIC_1pctCO2_TS_JJA, ARCTIC_4xext_TS_JJA], dim='time')
ARCTIC_1pctCO2_4xext_FSDS_JJA = xr.concat([ARCTIC_1pctCO2_FSDS_JJA, ARCTIC_4xext_FSDS_JJA], dim='time')
ARCTIC_1pctCO2_4xext_FSA_JJA = xr.concat([ARCTIC_1pctCO2_FSA_JJA, ARCTIC_4xext_FSA_JJA], dim='time')
ARCTIC_1pctCO2_4xext_FLDS_JJA = xr.concat([ARCTIC_1pctCO2_FLDS_JJA, ARCTIC_4xext_FLDS_JJA], dim='time')
ARCTIC_1pctCO2_4xext_FIRA_JJA = xr.concat([ARCTIC_1pctCO2_FIRA_JJA, ARCTIC_4xext_FIRA_JJA], dim='time')
ARCTIC_1pctCO2_4xext_FSH_JJA = xr.concat([ARCTIC_1pctCO2_FSH_JJA, ARCTIC_4xext_FSH_JJA], dim='time')
ARCTIC_1pctCO2_4xext_EFLX_LH_TOT_JJA = xr.concat([ARCTIC_1pctCO2_EFLX_LH_TOT_JJA, ARCTIC_4xext_EFLX_LH_TOT_JJA], dim='time')
ARCTIC_1pctCO2_4xext_FSM_JJA = xr.concat([ARCTIC_1pctCO2_FSM_JJA, ARCTIC_4xext_FSM_JJA], dim='time')
ARCTIC_1pctCO2_4xext_FICE_MELT_JJA = xr.concat([ARCTIC_1pctCO2_FICE_MELT_JJA, ARCTIC_4xext_FICE_MELT_JJA], dim='time')
ARCTIC_1pctCO2_4xext_FGR_JJA = xr.concat([ARCTIC_1pctCO2_FGR_JJA, ARCTIC_4xext_FGR_JJA], dim='time')
ARCTIC_1pctCO2_4xext_PHIS = xr.concat([ARCTIC_1pctCO2_PHIS, ARCTIC_4xext_PHIS], dim='time')

In [11]:
## Calculate variables
BG7_albedo_JJA = (BG7_FSDS_JJA - BG7_FSA_JJA) / BG7_FSDS_JJA
BG7_ME_JJA = BG7_FSM_JJA + BG7_FICE_MELT_JJA
#BG7_GHF_JJA = BG7_ME_JJA - BG7_FGR_JJA
BG7_GHF_JJA = BG7_ME_JJA - BG7_FSA_JJA + BG7_FIRA_JJA + BG7_FSH_JJA + BG7_EFLX_LH_TOT_JJA

f09_piControl_albedo_JJA = (f09_piControl_FSDS_JJA - f09_piControl_FSA_JJA) / f09_piControl_FSDS_JJA
f09_piControl_ME_JJA = f09_piControl_FSM_JJA + f09_piControl_FICE_MELT_JJA
f09_piControl_GHF_JJA = f09_piControl_ME_JJA - f09_piControl_FGR_JJA
f09_1pctCO2_4xext_albedo_JJA = (f09_1pctCO2_4xext_FSDS_JJA - f09_1pctCO2_4xext_FSA_JJA) / f09_1pctCO2_4xext_FSDS_JJA
f09_1pctCO2_4xext_ME_JJA = f09_1pctCO2_4xext_FSM_JJA + f09_1pctCO2_4xext_FICE_MELT_JJA
f09_1pctCO2_4xext_GHF_JJA = f09_1pctCO2_4xext_ME_JJA - f09_1pctCO2_4xext_FGR_JJA
f09_1pctCO2_4xext_elev = f09_1pctCO2_4xext_PHIS / g

f09_nohacks_piControl_albedo_JJA = (f09_nohacks_piControl_FSDS_JJA - f09_nohacks_piControl_FSA_JJA) / f09_nohacks_piControl_FSDS_JJA
f09_nohacks_piControl_ME_JJA = f09_nohacks_piControl_FSM_JJA + f09_nohacks_piControl_FICE_MELT_JJA
f09_nohacks_piControl_GHF_JJA = f09_nohacks_piControl_ME_JJA - f09_nohacks_piControl_FGR_JJA
f09_nohacks_1pctCO2_4xext_albedo_JJA = (f09_nohacks_1pctCO2_4xext_FSDS_JJA - f09_nohacks_1pctCO2_4xext_FSA_JJA) / f09_nohacks_1pctCO2_4xext_FSDS_JJA
f09_nohacks_1pctCO2_4xext_ME_JJA = f09_nohacks_1pctCO2_4xext_FSM_JJA + f09_nohacks_1pctCO2_4xext_FICE_MELT_JJA
f09_nohacks_1pctCO2_4xext_GHF_JJA = f09_nohacks_1pctCO2_4xext_ME_JJA - f09_nohacks_1pctCO2_4xext_FGR_JJA
f09_nohacks_1pctCO2_4xext_elev = f09_nohacks_1pctCO2_4xext_PHIS / g

ARCTIC_piControl_p4_albedo_JJA = (ARCTIC_piControl_p4_FSDS_JJA - ARCTIC_piControl_p4_FSA_JJA) / ARCTIC_piControl_p4_FSDS_JJA
ARCTIC_piControl_p4_ME_JJA = ARCTIC_piControl_p4_FSM_JJA + ARCTIC_piControl_p4_FICE_MELT_JJA
ARCTIC_piControl_p4_GHF_JJA = ARCTIC_piControl_p4_ME_JJA - ARCTIC_piControl_p4_FGR_JJA
ARCTIC_1pctCO2_4xext_albedo_JJA = (ARCTIC_1pctCO2_4xext_FSDS_JJA - ARCTIC_1pctCO2_4xext_FSA_JJA) / ARCTIC_1pctCO2_4xext_FSDS_JJA
ARCTIC_1pctCO2_4xext_ME_JJA = ARCTIC_1pctCO2_4xext_FSM_JJA + ARCTIC_1pctCO2_4xext_FICE_MELT_JJA
ARCTIC_1pctCO2_4xext_GHF_JJA = ARCTIC_1pctCO2_4xext_ME_JJA - ARCTIC_1pctCO2_4xext_FGR_JJA
ARCTIC_1pctCO2_4xext_elev = ARCTIC_1pctCO2_4xext_PHIS / g

### Calculate

In [16]:
## Average over the GrIS, also change signs
# F09M
BG7_GrIS_area = np.nansum(BG7_GrIS_mask * f09_area, axis=(1,2))
BG7_avg_albedo_JJA = (BG7_albedo_JJA * BG7_GrIS_mask * f09_area).sum(dim=['lat','lon']) / BG7_GrIS_area
BG7_avg_TREFHT_JJA = (BG7_TREFHT_JJA * BG7_GrIS_mask * f09_area).sum(dim=['lat','lon']) / BG7_GrIS_area - 273.15
BG7_avg_TS_JJA = (BG7_TS_JJA * BG7_GrIS_mask * f09_area).sum(dim=['lat','lon']) / BG7_GrIS_area - 273.15
BG7_avg_SWd_JJA = (BG7_FSDS_JJA * BG7_GrIS_mask * f09_area).sum(dim=['lat','lon']) / BG7_GrIS_area
BG7_avg_LWd_JJA = (BG7_FLDS_JJA * BG7_GrIS_mask * f09_area).sum(dim=['lat','lon']) / BG7_GrIS_area
BG7_avg_ME_JJA = (BG7_ME_JJA * BG7_GrIS_mask * f09_area).sum(dim=['lat','lon']) / BG7_GrIS_area
BG7_avg_SWn_JJA = (BG7_FSA_JJA * BG7_GrIS_mask * f09_area).sum(dim=['lat','lon']) / BG7_GrIS_area
BG7_avg_LWn_JJA = -(BG7_FIRA_JJA * BG7_GrIS_mask * f09_area).sum(dim=['lat','lon']) / BG7_GrIS_area
BG7_avg_SHF_JJA = -(BG7_FSH_JJA * BG7_GrIS_mask * f09_area).sum(dim=['lat','lon']) / BG7_GrIS_area
BG7_avg_LHF_JJA = -(BG7_EFLX_LH_TOT_JJA * BG7_GrIS_mask * f09_area).sum(dim=['lat','lon']) / BG7_GrIS_area
BG7_avg_GHF_JJA = (BG7_GHF_JJA * BG7_GrIS_mask * f09_area).sum(dim=['lat','lon']) / BG7_GrIS_area

f09_piControl_GrIS_area = np.nansum(f09_GrIS_mask_piControl * f09_area, axis=(1,2))
f09_piControl_avg_albedo_JJA = (f09_piControl_albedo_JJA * f09_GrIS_mask_piControl * f09_area).sum(dim=['lat','lon']) / f09_piControl_GrIS_area
#f09_piControl_avg_TREFHT_JJA = (f09_piControl_TREFHT_JJA * f09_GrIS_mask_piControl * f09_area).sum(dim=['lat','lon']) / f09_piControl_GrIS_area - 273.15
#f09_piControl_avg_TS_JJA = (f09_piControl_TS_JJA * f09_GrIS_mask_piControl * f09_area).sum(dim=['lat','lon']) / f09_piControl_GrIS_area - 273.15
f09_piControl_avg_SWd_JJA = (f09_piControl_FSDS_JJA * f09_GrIS_mask_piControl * f09_area).sum(dim=['lat','lon']) / f09_piControl_GrIS_area
f09_piControl_avg_LWd_JJA = (f09_piControl_FLDS_JJA * f09_GrIS_mask_piControl * f09_area).sum(dim=['lat','lon']) / f09_piControl_GrIS_area
f09_piControl_avg_ME_JJA = (f09_piControl_ME_JJA * f09_GrIS_mask_piControl * f09_area).sum(dim=['lat','lon']) / f09_piControl_GrIS_area
f09_piControl_avg_SWn_JJA = (f09_piControl_FSA_JJA * f09_GrIS_mask_piControl * f09_area).sum(dim=['lat','lon']) / f09_piControl_GrIS_area
f09_piControl_avg_LWn_JJA = -(f09_piControl_FIRA_JJA * f09_GrIS_mask_piControl * f09_area).sum(dim=['lat','lon']) / f09_piControl_GrIS_area
f09_piControl_avg_SHF_JJA = -(f09_piControl_FSH_JJA * f09_GrIS_mask_piControl * f09_area).sum(dim=['lat','lon']) / f09_piControl_GrIS_area
f09_piControl_avg_LHF_JJA = -(f09_piControl_EFLX_LH_TOT_JJA * f09_GrIS_mask_piControl * f09_area).sum(dim=['lat','lon']) / f09_piControl_GrIS_area
f09_piControl_avg_GHF_JJA = (f09_piControl_GHF_JJA * f09_GrIS_mask_piControl * f09_area).sum(dim=['lat','lon']) / f09_piControl_GrIS_area

f09_1pctCO2_4xext_GrIS_area = np.nansum(f09_GrIS_mask * f09_area, axis=(1,2))
f09_1pctCO2_4xext_avg_albedo_JJA = (f09_1pctCO2_4xext_albedo_JJA * f09_GrIS_mask * f09_area).sum(dim=['lat','lon']) / f09_1pctCO2_4xext_GrIS_area
f09_1pctCO2_4xext_avg_TREFHT_JJA = (f09_1pctCO2_4xext_TREFHT_JJA * f09_GrIS_mask * f09_area).sum(dim=['lat','lon']) / f09_1pctCO2_4xext_GrIS_area - 273.15
f09_1pctCO2_4xext_avg_TS_JJA = (f09_1pctCO2_4xext_TS_JJA * f09_GrIS_mask * f09_area).sum(dim=['lat','lon']) / f09_1pctCO2_4xext_GrIS_area - 273.15
f09_1pctCO2_4xext_avg_SWd_JJA = (f09_1pctCO2_4xext_FSDS_JJA * f09_GrIS_mask * f09_area).sum(dim=['lat','lon']) / f09_1pctCO2_4xext_GrIS_area
f09_1pctCO2_4xext_avg_LWd_JJA = (f09_1pctCO2_4xext_FLDS_JJA * f09_GrIS_mask * f09_area).sum(dim=['lat','lon']) / f09_1pctCO2_4xext_GrIS_area
f09_1pctCO2_4xext_avg_ME_JJA = (f09_1pctCO2_4xext_ME_JJA * f09_GrIS_mask * f09_area).sum(dim=['lat','lon']) / f09_1pctCO2_4xext_GrIS_area
f09_1pctCO2_4xext_avg_SWn_JJA = (f09_1pctCO2_4xext_FSA_JJA * f09_GrIS_mask * f09_area).sum(dim=['lat','lon']) / f09_1pctCO2_4xext_GrIS_area
f09_1pctCO2_4xext_avg_LWn_JJA = -(f09_1pctCO2_4xext_FIRA_JJA * f09_GrIS_mask * f09_area).sum(dim=['lat','lon']) / f09_1pctCO2_4xext_GrIS_area
f09_1pctCO2_4xext_avg_SHF_JJA = -(f09_1pctCO2_4xext_FSH_JJA * f09_GrIS_mask * f09_area).sum(dim=['lat','lon']) / f09_1pctCO2_4xext_GrIS_area
f09_1pctCO2_4xext_avg_LHF_JJA = -(f09_1pctCO2_4xext_EFLX_LH_TOT_JJA * f09_GrIS_mask * f09_area).sum(dim=['lat','lon']) / f09_1pctCO2_4xext_GrIS_area
f09_1pctCO2_4xext_avg_GHF_JJA = (f09_1pctCO2_4xext_GHF_JJA * f09_GrIS_mask * f09_area).sum(dim=['lat','lon']) / f09_1pctCO2_4xext_GrIS_area

#F09
f09_nohacks_piControl_GrIS_area = np.nansum(f09_nohacks_GrIS_mask_piControl * f09_area, axis=(1,2))
f09_nohacks_piControl_avg_albedo_JJA = (f09_nohacks_piControl_albedo_JJA * f09_nohacks_GrIS_mask_piControl * f09_area).sum(dim=['lat','lon']) / f09_nohacks_piControl_GrIS_area
f09_nohacks_piControl_avg_TREFHT_JJA = (f09_nohacks_piControl_TREFHT_JJA * f09_nohacks_GrIS_mask_piControl * f09_area).sum(dim=['lat','lon']) / f09_nohacks_piControl_GrIS_area - 273.15
f09_nohacks_piControl_avg_TS_JJA = (f09_nohacks_piControl_TS_JJA * f09_nohacks_GrIS_mask_piControl * f09_area).sum(dim=['lat','lon']) / f09_nohacks_piControl_GrIS_area - 273.15
f09_nohacks_piControl_avg_SWd_JJA = (f09_nohacks_piControl_FSDS_JJA * f09_nohacks_GrIS_mask_piControl * f09_area).sum(dim=['lat','lon']) / f09_nohacks_piControl_GrIS_area
f09_nohacks_piControl_avg_LWd_JJA = (f09_nohacks_piControl_FLDS_JJA * f09_nohacks_GrIS_mask_piControl * f09_area).sum(dim=['lat','lon']) / f09_nohacks_piControl_GrIS_area
f09_nohacks_piControl_avg_ME_JJA = (f09_nohacks_piControl_ME_JJA * f09_nohacks_GrIS_mask_piControl * f09_area).sum(dim=['lat','lon']) / f09_nohacks_piControl_GrIS_area
f09_nohacks_piControl_avg_SWn_JJA = (f09_nohacks_piControl_FSA_JJA * f09_nohacks_GrIS_mask_piControl * f09_area).sum(dim=['lat','lon']) / f09_nohacks_piControl_GrIS_area
f09_nohacks_piControl_avg_LWn_JJA = -(f09_nohacks_piControl_FIRA_JJA * f09_nohacks_GrIS_mask_piControl * f09_area).sum(dim=['lat','lon']) / f09_nohacks_piControl_GrIS_area
f09_nohacks_piControl_avg_SHF_JJA = -(f09_nohacks_piControl_FSH_JJA * f09_nohacks_GrIS_mask_piControl * f09_area).sum(dim=['lat','lon']) / f09_nohacks_piControl_GrIS_area
f09_nohacks_piControl_avg_LHF_JJA = -(f09_nohacks_piControl_EFLX_LH_TOT_JJA * f09_nohacks_GrIS_mask_piControl * f09_area).sum(dim=['lat','lon']) / f09_nohacks_piControl_GrIS_area
f09_nohacks_piControl_avg_GHF_JJA = (f09_nohacks_piControl_GHF_JJA * f09_nohacks_GrIS_mask_piControl * f09_area).sum(dim=['lat','lon']) / f09_nohacks_piControl_GrIS_area

f09_nohacks_1pctCO2_4xext_GrIS_area = np.nansum(f09_nohacks_GrIS_mask * f09_area, axis=(1,2))
f09_nohacks_1pctCO2_4xext_avg_albedo_JJA = (f09_nohacks_1pctCO2_4xext_albedo_JJA * f09_nohacks_GrIS_mask * f09_area).sum(dim=['lat','lon']) / f09_nohacks_1pctCO2_4xext_GrIS_area
f09_nohacks_1pctCO2_4xext_avg_TREFHT_JJA = (f09_nohacks_1pctCO2_4xext_TREFHT_JJA * f09_nohacks_GrIS_mask * f09_area).sum(dim=['lat','lon']) / f09_nohacks_1pctCO2_4xext_GrIS_area - 273.15
f09_nohacks_1pctCO2_4xext_avg_TS_JJA = (f09_nohacks_1pctCO2_4xext_TS_JJA * f09_nohacks_GrIS_mask * f09_area).sum(dim=['lat','lon']) / f09_nohacks_1pctCO2_4xext_GrIS_area - 273.15
f09_nohacks_1pctCO2_4xext_avg_SWd_JJA = (f09_nohacks_1pctCO2_4xext_FSDS_JJA * f09_nohacks_GrIS_mask * f09_area).sum(dim=['lat','lon']) / f09_nohacks_1pctCO2_4xext_GrIS_area
f09_nohacks_1pctCO2_4xext_avg_LWd_JJA = (f09_nohacks_1pctCO2_4xext_FLDS_JJA * f09_nohacks_GrIS_mask * f09_area).sum(dim=['lat','lon']) / f09_nohacks_1pctCO2_4xext_GrIS_area
f09_nohacks_1pctCO2_4xext_avg_ME_JJA = (f09_nohacks_1pctCO2_4xext_ME_JJA * f09_nohacks_GrIS_mask * f09_area).sum(dim=['lat','lon']) / f09_nohacks_1pctCO2_4xext_GrIS_area
f09_nohacks_1pctCO2_4xext_avg_SWn_JJA = (f09_nohacks_1pctCO2_4xext_FSA_JJA * f09_nohacks_GrIS_mask * f09_area).sum(dim=['lat','lon']) / f09_nohacks_1pctCO2_4xext_GrIS_area
f09_nohacks_1pctCO2_4xext_avg_LWn_JJA = -(f09_nohacks_1pctCO2_4xext_FIRA_JJA * f09_nohacks_GrIS_mask * f09_area).sum(dim=['lat','lon']) / f09_nohacks_1pctCO2_4xext_GrIS_area
f09_nohacks_1pctCO2_4xext_avg_SHF_JJA = -(f09_nohacks_1pctCO2_4xext_FSH_JJA * f09_nohacks_GrIS_mask * f09_area).sum(dim=['lat','lon']) / f09_nohacks_1pctCO2_4xext_GrIS_area
f09_nohacks_1pctCO2_4xext_avg_LHF_JJA = -(f09_nohacks_1pctCO2_4xext_EFLX_LH_TOT_JJA * f09_nohacks_GrIS_mask * f09_area).sum(dim=['lat','lon']) / f09_nohacks_1pctCO2_4xext_GrIS_area
f09_nohacks_1pctCO2_4xext_avg_GHF_JJA = (f09_nohacks_1pctCO2_4xext_GHF_JJA * f09_nohacks_GrIS_mask * f09_area).sum(dim=['lat','lon']) / f09_nohacks_1pctCO2_4xext_GrIS_area

#ARCTIC
ARCTIC_piControl_p4_GrIS_area = np.nansum(ARCTIC_GrIS_mask_piControl[-80:,:] * ARCTIC_area, axis=1)
ARCTIC_piControl_p4_avg_albedo_JJA = (ARCTIC_piControl_p4_albedo_JJA * ARCTIC_GrIS_mask_piControl[-80:,:] * ARCTIC_area).sum(dim=['lndgrid']) / ARCTIC_piControl_p4_GrIS_area
ARCTIC_piControl_p4_avg_TREFHT_JJA = (ARCTIC_piControl_p4_TREFHT_JJA * ARCTIC_GrIS_mask_piControl[-80:,:] * ARCTIC_area).sum(dim=['ncol']) / ARCTIC_piControl_p4_GrIS_area - 273.15
ARCTIC_piControl_p4_avg_TS_JJA = (ARCTIC_piControl_p4_TS_JJA * ARCTIC_GrIS_mask_piControl[-80:,:] * ARCTIC_area).sum(dim=['ncol']) / ARCTIC_piControl_p4_GrIS_area - 273.15
ARCTIC_piControl_p4_avg_SWd_JJA = (ARCTIC_piControl_p4_FSDS_JJA * ARCTIC_GrIS_mask_piControl[-80:,:] * ARCTIC_area).sum(dim=['lndgrid']) / ARCTIC_piControl_p4_GrIS_area
ARCTIC_piControl_p4_avg_LWd_JJA = (ARCTIC_piControl_p4_FLDS_JJA * ARCTIC_GrIS_mask_piControl[-80:,:] * ARCTIC_area).sum(dim=['lndgrid']) / ARCTIC_piControl_p4_GrIS_area
ARCTIC_piControl_p4_avg_ME_JJA = (ARCTIC_piControl_p4_ME_JJA * ARCTIC_GrIS_mask_piControl[-80:,:] * ARCTIC_area).sum(dim=['lndgrid']) / ARCTIC_piControl_p4_GrIS_area
ARCTIC_piControl_p4_avg_SWn_JJA = (ARCTIC_piControl_p4_FSA_JJA * ARCTIC_GrIS_mask_piControl[-80:,:] * ARCTIC_area).sum(dim=['lndgrid']) / ARCTIC_piControl_p4_GrIS_area
ARCTIC_piControl_p4_avg_LWn_JJA = -(ARCTIC_piControl_p4_FIRA_JJA * ARCTIC_GrIS_mask_piControl[-80:,:] * ARCTIC_area).sum(dim=['lndgrid']) / ARCTIC_piControl_p4_GrIS_area
ARCTIC_piControl_p4_avg_SHF_JJA = -(ARCTIC_piControl_p4_FSH_JJA * ARCTIC_GrIS_mask_piControl[-80:,:] * ARCTIC_area).sum(dim=['lndgrid']) / ARCTIC_piControl_p4_GrIS_area
ARCTIC_piControl_p4_avg_LHF_JJA = -(ARCTIC_piControl_p4_EFLX_LH_TOT_JJA * ARCTIC_GrIS_mask_piControl[-80:,:] * ARCTIC_area).sum(dim=['lndgrid']) / ARCTIC_piControl_p4_GrIS_area
ARCTIC_piControl_p4_avg_GHF_JJA = (ARCTIC_piControl_p4_GHF_JJA * ARCTIC_GrIS_mask_piControl[-80:,:] * ARCTIC_area).sum(dim=['lndgrid']) / ARCTIC_piControl_p4_GrIS_area

ARCTIC_1pctCO2_4xext_GrIS_area = np.nansum(ARCTIC_GrIS_mask * ARCTIC_area, axis=1)
ARCTIC_1pctCO2_4xext_avg_albedo_JJA = (ARCTIC_1pctCO2_4xext_albedo_JJA * ARCTIC_GrIS_mask * ARCTIC_area).sum(dim=['lndgrid']) / ARCTIC_1pctCO2_4xext_GrIS_area
ARCTIC_1pctCO2_4xext_avg_TREFHT_JJA = (ARCTIC_1pctCO2_4xext_TREFHT_JJA * ARCTIC_GrIS_mask * ARCTIC_area).sum(dim=['ncol']) / ARCTIC_1pctCO2_4xext_GrIS_area - 273.15
ARCTIC_1pctCO2_4xext_avg_TS_JJA = (ARCTIC_1pctCO2_4xext_TS_JJA * ARCTIC_GrIS_mask * ARCTIC_area).sum(dim=['ncol']) / ARCTIC_1pctCO2_4xext_GrIS_area - 273.15
ARCTIC_1pctCO2_4xext_avg_SWd_JJA = (ARCTIC_1pctCO2_4xext_FSDS_JJA * ARCTIC_GrIS_mask * ARCTIC_area).sum(dim=['lndgrid']) / ARCTIC_1pctCO2_4xext_GrIS_area
ARCTIC_1pctCO2_4xext_avg_LWd_JJA = (ARCTIC_1pctCO2_4xext_FLDS_JJA * ARCTIC_GrIS_mask * ARCTIC_area).sum(dim=['lndgrid']) / ARCTIC_1pctCO2_4xext_GrIS_area
ARCTIC_1pctCO2_4xext_avg_ME_JJA = (ARCTIC_1pctCO2_4xext_ME_JJA * ARCTIC_GrIS_mask * ARCTIC_area).sum(dim=['lndgrid']) / ARCTIC_1pctCO2_4xext_GrIS_area
ARCTIC_1pctCO2_4xext_avg_SWn_JJA = (ARCTIC_1pctCO2_4xext_FSA_JJA * ARCTIC_GrIS_mask * ARCTIC_area).sum(dim=['lndgrid']) / ARCTIC_1pctCO2_4xext_GrIS_area
ARCTIC_1pctCO2_4xext_avg_LWn_JJA = -(ARCTIC_1pctCO2_4xext_FIRA_JJA * ARCTIC_GrIS_mask * ARCTIC_area).sum(dim=['lndgrid']) / ARCTIC_1pctCO2_4xext_GrIS_area
ARCTIC_1pctCO2_4xext_avg_SHF_JJA = -(ARCTIC_1pctCO2_4xext_FSH_JJA * ARCTIC_GrIS_mask * ARCTIC_area).sum(dim=['lndgrid']) / ARCTIC_1pctCO2_4xext_GrIS_area
ARCTIC_1pctCO2_4xext_avg_LHF_JJA = -(ARCTIC_1pctCO2_4xext_EFLX_LH_TOT_JJA * ARCTIC_GrIS_mask * ARCTIC_area).sum(dim=['lndgrid']) / ARCTIC_1pctCO2_4xext_GrIS_area
ARCTIC_1pctCO2_4xext_avg_GHF_JJA = (ARCTIC_1pctCO2_4xext_GHF_JJA * ARCTIC_GrIS_mask * ARCTIC_area).sum(dim=['lndgrid']) / ARCTIC_1pctCO2_4xext_GrIS_area

### Result

In [23]:
# Albedo
print('Albedo')
print('')

print('F09M(P0): ', BG7_avg_albedo_JJA[129:149].mean().round(2).values)
print('F09(P0): ', f09_nohacks_piControl_avg_albedo_JJA[-20:].mean().round(2).values)
print('ARCTIC(P0): ', ARCTIC_piControl_p4_avg_albedo_JJA[-20:].mean().round(2).values)
print('')

print('F09M(P1): ', f09_1pctCO2_4xext_avg_albedo_JJA[130:150].mean().round(2).values)
print('F09(P1): ', f09_nohacks_1pctCO2_4xext_avg_albedo_JJA[130:150].mean().round(2).values)
print('ARCTIC(P1): ', ARCTIC_1pctCO2_4xext_avg_albedo_JJA[130:150].mean().round(2).values)
print('')

print('F09M(P2): ', f09_1pctCO2_4xext_avg_albedo_JJA[230:250].mean().round(2).values)
print('F09(P2): ', f09_nohacks_1pctCO2_4xext_avg_albedo_JJA[230:250].mean().round(2).values)
print('ARCTIC(P2): ', ARCTIC_1pctCO2_4xext_avg_albedo_JJA[230:250].mean().round(2).values)
print('')

print('F09M(P3): ', f09_1pctCO2_4xext_avg_albedo_JJA[330:350].mean().round(2).values)
print('F09(P3): ', f09_nohacks_1pctCO2_4xext_avg_albedo_JJA[330:350].mean().round(2).values)
print('ARCTIC(P3): ', ARCTIC_1pctCO2_4xext_avg_albedo_JJA[330:350].mean().round(2).values)

Albedo

F09M(P0):  0.78
F09(P0):  0.78
ARCTIC(P0):  0.78

F09M(P1):  0.71
F09(P1):  0.72
ARCTIC(P1):  0.73

F09M(P2):  0.64
F09(P2):  0.66
ARCTIC(P2):  0.68

F09M(P3):  0.61
F09(P3):  0.62
ARCTIC(P3):  0.65


In [17]:
# TREFHT
print('TREFHT')
print('')

print('F09M(P0): ', BG7_avg_TREFHT_JJA[129:149].mean().round(2).values)
print('F09(P0): ', f09_nohacks_piControl_avg_TREFHT_JJA[-20:].mean().round(2).values)
print('ARCTIC(P0): ', ARCTIC_piControl_p4_avg_TREFHT_JJA[-20:].mean().round(2).values)
print('')

print('F09M(P1): ', f09_1pctCO2_4xext_avg_TREFHT_JJA[130:150].mean().round(2).values)
print('F09(P1): ', f09_nohacks_1pctCO2_4xext_avg_TREFHT_JJA[130:150].mean().round(2).values)
print('ARCTIC(P1): ', ARCTIC_1pctCO2_4xext_avg_TREFHT_JJA[130:150].mean().round(2).values)
print('')

print('F09M(P2): ', f09_1pctCO2_4xext_avg_TREFHT_JJA[230:250].mean().round(2).values)
print('F09(P2): ', f09_nohacks_1pctCO2_4xext_avg_TREFHT_JJA[230:250].mean().round(2).values)
print('ARCTIC(P2): ', ARCTIC_1pctCO2_4xext_avg_TREFHT_JJA[230:250].mean().round(2).values)
print('')

print('F09M(P3): ', f09_1pctCO2_4xext_avg_TREFHT_JJA[330:350].mean().round(2).values)
print('F09(P3): ', f09_nohacks_1pctCO2_4xext_avg_TREFHT_JJA[330:350].mean().round(2).values)
print('ARCTIC(P3): ', ARCTIC_1pctCO2_4xext_avg_TREFHT_JJA[330:350].mean().round(2).values)

TREFHT

F09M(P0):  -7.0
F09(P0):  -7.05
ARCTIC(P0):  -7.34

F09M(P1):  -1.22
F09(P1):  -1.11
ARCTIC(P1):  -1.88

F09M(P2):  0.5
F09(P2):  0.43
ARCTIC(P2):  -0.22

F09M(P3):  1.03
F09(P3):  1.21
ARCTIC(P3):  0.79


In [18]:
# TS
print('TS')
print('')

print('F09M(P0): ', BG7_avg_TS_JJA[129:149].mean().round(2).values)
print('F09(P0): ', f09_nohacks_piControl_avg_TS_JJA[-20:].mean().round(2).values)
print('ARCTIC(P0): ', ARCTIC_piControl_p4_avg_TS_JJA[-20:].mean().round(2).values)
print('')

print('F09M(P1): ', f09_1pctCO2_4xext_avg_TS_JJA[130:150].mean().round(2).values)
print('F09(P1): ', f09_nohacks_1pctCO2_4xext_avg_TS_JJA[130:150].mean().round(2).values)
print('ARCTIC(P1): ', ARCTIC_1pctCO2_4xext_avg_TS_JJA[130:150].mean().round(2).values)
print('')

print('F09M(P2): ', f09_1pctCO2_4xext_avg_TS_JJA[230:250].mean().round(2).values)
print('F09(P2): ', f09_nohacks_1pctCO2_4xext_avg_TS_JJA[230:250].mean().round(2).values)
print('ARCTIC(P2): ', ARCTIC_1pctCO2_4xext_avg_TS_JJA[230:250].mean().round(2).values)
print('')

print('F09M(P3): ', f09_1pctCO2_4xext_avg_TS_JJA[330:350].mean().round(2).values)
print('F09(P3): ', f09_nohacks_1pctCO2_4xext_avg_TS_JJA[330:350].mean().round(2).values)
print('ARCTIC(P3): ', ARCTIC_1pctCO2_4xext_avg_TS_JJA[330:350].mean().round(2).values)

TS

F09M(P0):  -7.83
F09(P0):  -7.83
ARCTIC(P0):  -8.42

F09M(P1):  -2.14
F09(P1):  -2.15
ARCTIC(P1):  -3.1

F09M(P2):  -0.74
F09(P2):  -0.85
ARCTIC(P2):  -1.74

F09M(P3):  -0.31
F09(P3):  -0.2
ARCTIC(P3):  -0.96


In [26]:
# SWd
print('SWd')
print('')

print('F09M(P0): ', BG7_avg_SWd_JJA[129:149].mean().round(2).values)
print('F09(P0): ', f09_nohacks_piControl_avg_SWd_JJA[-20:].mean().round(2).values)
print('ARCTIC(P0): ', ARCTIC_piControl_p4_avg_SWd_JJA[-20:].mean().round(2).values)
print('')

print('F09M(P1): ', f09_1pctCO2_4xext_avg_SWd_JJA[130:150].mean().round(2).values)
print('F09(P1): ', f09_nohacks_1pctCO2_4xext_avg_SWd_JJA[130:150].mean().round(2).values)
print('ARCTIC(P1): ', ARCTIC_1pctCO2_4xext_avg_SWd_JJA[130:150].mean().round(2).values)
print('')

print('F09M(P2): ', f09_1pctCO2_4xext_avg_SWd_JJA[230:250].mean().round(2).values)
print('F09(P2): ', f09_nohacks_1pctCO2_4xext_avg_SWd_JJA[230:250].mean().round(2).values)
print('ARCTIC(P2): ', ARCTIC_1pctCO2_4xext_avg_SWd_JJA[230:250].mean().round(2).values)
print('')

print('F09M(P3): ', f09_1pctCO2_4xext_avg_SWd_JJA[330:350].mean().round(2).values)
print('F09(P3): ', f09_nohacks_1pctCO2_4xext_avg_SWd_JJA[330:350].mean().round(2).values)
print('ARCTIC(P3): ', ARCTIC_1pctCO2_4xext_avg_SWd_JJA[330:350].mean().round(2).values)

SWd

F09M(P0):  288.33
F09(P0):  286.41
ARCTIC(P0):  294.3

F09M(P1):  261.98
F09(P1):  261.66
ARCTIC(P1):  272.6

F09M(P2):  255.86
F09(P2):  254.92
ARCTIC(P2):  266.56

F09M(P3):  250.94
F09(P3):  253.4
ARCTIC(P3):  263.98


In [27]:
# LWd
print('LWd')
print('')

print('F09M(P0): ', BG7_avg_LWd_JJA[129:149].mean().round(2).values)
print('F09(P0): ', f09_nohacks_piControl_avg_LWd_JJA[-20:].mean().round(2).values)
print('ARCTIC(P0): ', ARCTIC_piControl_p4_avg_LWd_JJA[-20:].mean().round(2).values)
print('')

print('F09M(P1): ', f09_1pctCO2_4xext_avg_LWd_JJA[130:150].mean().round(2).values)
print('F09(P1): ', f09_nohacks_1pctCO2_4xext_avg_LWd_JJA[130:150].mean().round(2).values)
print('ARCTIC(P1): ', ARCTIC_1pctCO2_4xext_avg_LWd_JJA[130:150].mean().round(2).values)
print('')

print('F09M(P2): ', f09_1pctCO2_4xext_avg_LWd_JJA[230:250].mean().round(2).values)
print('F09(P2): ', f09_nohacks_1pctCO2_4xext_avg_LWd_JJA[230:250].mean().round(2).values)
print('ARCTIC(P2): ', ARCTIC_1pctCO2_4xext_avg_LWd_JJA[230:250].mean().round(2).values)
print('')

print('F09M(P3): ', f09_1pctCO2_4xext_avg_LWd_JJA[330:350].mean().round(2).values)
print('F09(P3): ', f09_nohacks_1pctCO2_4xext_avg_LWd_JJA[330:350].mean().round(2).values)
print('ARCTIC(P3): ', ARCTIC_1pctCO2_4xext_avg_LWd_JJA[330:350].mean().round(2).values)

LWd

F09M(P0):  231.02
F09(P0):  232.62
ARCTIC(P0):  224.11

F09M(P1):  267.72
F09(P1):  268.45
ARCTIC(P1):  257.81

F09M(P2):  276.14
F09(P2):  276.81
ARCTIC(P2):  266.65

F09M(P3):  280.34
F09(P3):  280.69
ARCTIC(P3):  271.55


In [30]:
# ME
print('ME')
print('')

print('F09M(P0): ', BG7_avg_ME_JJA[129:149].mean().round(2).values)
print('F09(P0): ', f09_nohacks_piControl_avg_ME_JJA[-20:].mean().round(2).values)
print('ARCTIC(P0): ', ARCTIC_piControl_p4_avg_ME_JJA[-20:].mean().round(2).values)
print('')

print('F09M(P1): ', f09_1pctCO2_4xext_avg_ME_JJA[130:150].mean().round(2).values)
print('F09(P1): ', f09_nohacks_1pctCO2_4xext_avg_ME_JJA[130:150].mean().round(2).values)
print('ARCTIC(P1): ', ARCTIC_1pctCO2_4xext_avg_ME_JJA[130:150].mean().round(2).values)
print('')

print('F09M(P2): ', f09_1pctCO2_4xext_avg_ME_JJA[230:250].mean().round(2).values)
print('F09(P2): ', f09_nohacks_1pctCO2_4xext_avg_ME_JJA[230:250].mean().round(2).values)
print('ARCTIC(P2): ', ARCTIC_1pctCO2_4xext_avg_ME_JJA[230:250].mean().round(2).values)
print('')

print('F09M(P3): ', f09_1pctCO2_4xext_avg_ME_JJA[330:350].mean().round(2).values)
print('F09(P3): ', f09_nohacks_1pctCO2_4xext_avg_ME_JJA[330:350].mean().round(2).values)
print('ARCTIC(P3): ', ARCTIC_1pctCO2_4xext_avg_ME_JJA[330:350].mean().round(2).values)

ME

F09M(P0):  8.78
F09(P0):  8.46
ARCTIC(P0):  8.38

F09M(P1):  40.33
F09(P1):  43.37
ARCTIC(P1):  35.69

F09M(P2):  71.0
F09(P2):  68.3
ARCTIC(P2):  59.66

F09M(P3):  84.64
F09(P3):  83.62
ARCTIC(P3):  79.97


In [28]:
# SWn
print('SWn')
print('')

print('F09M(P0): ', BG7_avg_SWn_JJA[129:149].mean().round(2).values)
print('F09(P0): ', f09_nohacks_piControl_avg_SWn_JJA[-20:].mean().round(2).values)
print('ARCTIC(P0): ', ARCTIC_piControl_p4_avg_SWn_JJA[-20:].mean().round(2).values)
print('')

print('F09M(P1): ', f09_1pctCO2_4xext_avg_SWn_JJA[130:150].mean().round(2).values)
print('F09(P1): ', f09_nohacks_1pctCO2_4xext_avg_SWn_JJA[130:150].mean().round(2).values)
print('ARCTIC(P1): ', ARCTIC_1pctCO2_4xext_avg_SWn_JJA[130:150].mean().round(2).values)
print('')

print('F09M(P2): ', f09_1pctCO2_4xext_avg_SWn_JJA[230:250].mean().round(2).values)
print('F09(P2): ', f09_nohacks_1pctCO2_4xext_avg_SWn_JJA[230:250].mean().round(2).values)
print('ARCTIC(P2): ', ARCTIC_1pctCO2_4xext_avg_SWn_JJA[230:250].mean().round(2).values)
print('')

print('F09M(P3): ', f09_1pctCO2_4xext_avg_SWn_JJA[330:350].mean().round(2).values)
print('F09(P3): ', f09_nohacks_1pctCO2_4xext_avg_SWn_JJA[330:350].mean().round(2).values)
print('ARCTIC(P3): ', ARCTIC_1pctCO2_4xext_avg_SWn_JJA[330:350].mean().round(2).values)

SWn

F09M(P0):  64.01
F09(P0):  62.06
ARCTIC(P0):  64.88

F09M(P1):  73.48
F09(P1):  72.78
ARCTIC(P1):  73.53

F09M(P2):  89.46
F09(P2):  84.85
ARCTIC(P2):  83.18

F09M(P3):  94.83
F09(P3):  92.8
ARCTIC(P3):  91.58


In [29]:
# LWn
print('LWn')
print('')

print('F09M(P0): ', BG7_avg_LWn_JJA[129:149].mean().round(2).values)
print('F09(P0): ', f09_nohacks_piControl_avg_LWn_JJA[-20:].mean().round(2).values)
print('ARCTIC(P0): ', ARCTIC_piControl_p4_avg_LWn_JJA[-20:].mean().round(2).values)
print('')

print('F09M(P1): ', f09_1pctCO2_4xext_avg_LWn_JJA[130:150].mean().round(2).values)
print('F09(P1): ', f09_nohacks_1pctCO2_4xext_avg_LWn_JJA[130:150].mean().round(2).values)
print('ARCTIC(P1): ', ARCTIC_1pctCO2_4xext_avg_LWn_JJA[130:150].mean().round(2).values)
print('')

print('F09M(P2): ', f09_1pctCO2_4xext_avg_LWn_JJA[230:250].mean().round(2).values)
print('F09(P2): ', f09_nohacks_1pctCO2_4xext_avg_LWn_JJA[230:250].mean().round(2).values)
print('ARCTIC(P2): ', ARCTIC_1pctCO2_4xext_avg_LWn_JJA[230:250].mean().round(2).values)
print('')

print('F09M(P3): ', f09_1pctCO2_4xext_avg_LWn_JJA[330:350].mean().round(2).values)
print('F09(P3): ', f09_nohacks_1pctCO2_4xext_avg_LWn_JJA[330:350].mean().round(2).values)
print('ARCTIC(P3): ', ARCTIC_1pctCO2_4xext_avg_LWn_JJA[330:350].mean().round(2).values)

LWn

F09M(P0):  -50.61
F09(P0):  -49.03
ARCTIC(P0):  -55.26

F09M(P1):  -38.17
F09(P1):  -37.3
ARCTIC(P1):  -44.01

F09M(P2):  -36.06
F09(P2):  -34.85
ARCTIC(P2):  -41.24

F09M(P3):  -34.05
F09(P3):  -34.13
ARCTIC(P3):  -39.86


In [31]:
# SHF
print('SHF')
print('')

print('F09M(P0): ', BG7_avg_SHF_JJA[129:149].mean().round(2).values)
print('F09(P0): ', f09_nohacks_piControl_avg_SHF_JJA[-20:].mean().round(2).values)
print('ARCTIC(P0): ', ARCTIC_piControl_p4_avg_SHF_JJA[-20:].mean().round(2).values)
print('')

print('F09M(P1): ', f09_1pctCO2_4xext_avg_SHF_JJA[130:150].mean().round(2).values)
print('F09(P1): ', f09_nohacks_1pctCO2_4xext_avg_SHF_JJA[130:150].mean().round(2).values)
print('ARCTIC(P1): ', ARCTIC_1pctCO2_4xext_avg_SHF_JJA[130:150].mean().round(2).values)
print('')

print('F09M(P2): ', f09_1pctCO2_4xext_avg_SHF_JJA[230:250].mean().round(2).values)
print('F09(P2): ', f09_nohacks_1pctCO2_4xext_avg_SHF_JJA[230:250].mean().round(2).values)
print('ARCTIC(P2): ', ARCTIC_1pctCO2_4xext_avg_SHF_JJA[230:250].mean().round(2).values)
print('')

print('F09M(P3): ', f09_1pctCO2_4xext_avg_SHF_JJA[330:350].mean().round(2).values)
print('F09(P3): ', f09_nohacks_1pctCO2_4xext_avg_SHF_JJA[330:350].mean().round(2).values)
print('ARCTIC(P3): ', ARCTIC_1pctCO2_4xext_avg_SHF_JJA[330:350].mean().round(2).values)

SHF

F09M(P0):  4.82
F09(P0):  4.46
ARCTIC(P0):  7.13

F09M(P1):  8.35
F09(P1):  10.4
ARCTIC(P1):  11.22

F09M(P2):  14.24
F09(P2):  14.73
ARCTIC(P2):  17.61

F09M(P3):  16.05
F09(P3):  16.67
ARCTIC(P3):  23.25


In [20]:
# LHF
print('LHF')
print('')

print('F09M(P0): ', BG7_avg_LHF_JJA[129:149].mean().round(2).values)
print('F09(P0): ', f09_nohacks_piControl_avg_LHF_JJA[-20:].mean().round(2).values)
print('ARCTIC(P0): ', ARCTIC_piControl_p4_avg_LHF_JJA[-20:].mean().round(2).values)
print('')

print('F09M(P1): ', f09_1pctCO2_4xext_avg_LHF_JJA[130:150].mean().round(2).values)
print('F09(P1): ', f09_nohacks_1pctCO2_4xext_avg_LHF_JJA[130:150].mean().round(2).values)
print('ARCTIC(P1): ', ARCTIC_1pctCO2_4xext_avg_LHF_JJA[130:150].mean().round(2).values)
print('')

print('F09M(P2): ', f09_1pctCO2_4xext_avg_LHF_JJA[230:250].mean().round(2).values)
print('F09(P2): ', f09_nohacks_1pctCO2_4xext_avg_LHF_JJA[230:250].mean().round(2).values)
print('ARCTIC(P2): ', ARCTIC_1pctCO2_4xext_avg_LHF_JJA[230:250].mean().round(2).values)
print('')

print('F09M(P3): ', f09_1pctCO2_4xext_avg_LHF_JJA[330:350].mean().round(2).values)
print('F09(P3): ', f09_nohacks_1pctCO2_4xext_avg_LHF_JJA[330:350].mean().round(2).values)
print('ARCTIC(P3): ', ARCTIC_1pctCO2_4xext_avg_LHF_JJA[330:350].mean().round(2).values)

LHF

F09M(P0):  -8.26
F09(P0):  -7.98
ARCTIC(P0):  -7.54

F09M(P1):  -6.91
F09(P1):  -5.43
ARCTIC(P1):  -7.13

F09M(P2):  -2.72
F09(P2):  -2.29
ARCTIC(P2):  -3.64

F09M(P3):  -0.59
F09(P3):  0.05
ARCTIC(P3):  -0.3


In [21]:
# GHF
print('GHF')
print('')

print('F09M(P0): ', BG7_avg_GHF_JJA[129:149].mean().round(2).values)
print('F09(P0): ', f09_nohacks_piControl_avg_GHF_JJA[-20:].mean().round(2).values)
print('ARCTIC(P0): ', ARCTIC_piControl_p4_avg_GHF_JJA[-20:].mean().round(2).values)
print('')

print('F09M(P1): ', f09_1pctCO2_4xext_avg_GHF_JJA[130:150].mean().round(2).values)
print('F09(P1): ', f09_nohacks_1pctCO2_4xext_avg_GHF_JJA[130:150].mean().round(2).values)
print('ARCTIC(P1): ', ARCTIC_1pctCO2_4xext_avg_GHF_JJA[130:150].mean().round(2).values)
print('')

print('F09M(P2): ', f09_1pctCO2_4xext_avg_GHF_JJA[230:250].mean().round(2).values)
print('F09(P2): ', f09_nohacks_1pctCO2_4xext_avg_GHF_JJA[230:250].mean().round(2).values)
print('ARCTIC(P2): ', ARCTIC_1pctCO2_4xext_avg_GHF_JJA[230:250].mean().round(2).values)
print('')

print('F09M(P3): ', f09_1pctCO2_4xext_avg_GHF_JJA[330:350].mean().round(2).values)
print('F09(P3): ', f09_nohacks_1pctCO2_4xext_avg_GHF_JJA[330:350].mean().round(2).values)
print('ARCTIC(P3): ', ARCTIC_1pctCO2_4xext_avg_GHF_JJA[330:350].mean().round(2).values)

GHF

F09M(P0):  -1.18
F09(P0):  -1.06
ARCTIC(P0):  -0.82

F09M(P1):  3.58
F09(P1):  2.9
ARCTIC(P1):  2.07

F09M(P2):  6.07
F09(P2):  5.85
ARCTIC(P2):  3.76

F09M(P3):  8.39
F09(P3):  8.23
ARCTIC(P3):  5.3
